# AI-based Surveying the Impact of Environmental, Climatic, Economic and Demographic Conditions on the Epidemic Outbreak Rate of COVID-19

$Mehryar{\,}Majd$, $Ramin{\,}Mousa$ \& $Arsalan{\,}Mousazadeh$

__Abstract:__ This study aims to have an analytical and predictive approach to the growth rate and spread of the first wave of COVID-19 infection and mainly focuses on its dependency on other global factors in affected countries via Artificial Intelligence-based processing of fatality data/statistics. Fatality, infected, and recovered cases in the form of the time-series data was acquired through the World Health Organization (WHO) & the Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE). Recorded health data subsequently was analyzed by several Deep Neural Network (DNN) models to create a better intuition of epidemic outbreak rate of COVID-19. Monitoring spreading trend changes becomes more feasible via continuous observation of statistical pattern and integrating the other datasets with more global features count as factors influencing the local conditions on historical data conveys pivotal information concerning learning the pattern of epidemic progression. In this study, attention-based Sequence-to-Sequence (Seq2Seq) models empowered by Long-Short Term Memory (LSTM) and Gated Recurrent Unit (GRU) units, were deployed to predict mortality rate. Ultimately these analyses are utilized to monitor infection behaviour on time-series data and were subsequently assessed for their performance to predict the behaviour of the first wave of COVID-19 outbreak rate intensely. Demographic & environmental conditions have likely been influenced significantly to shape the reliable predictive learning algorithms for long-duration risk of COVID-19 pandemic.

### Feature engineering on __weather data__
__Data Cleaning & Preprocessing__

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from matplotlib import pyplot as plt
import os

In [2]:
df_covid_19 = pd.read_csv("novel-corona-virus/covid_19_data.csv")
df_covid_19['Date'] = pd.to_datetime(df_covid_19['ObservationDate'])
df_covid_19['Outbreak'] = 'COVID_2019'
df_sars_03 = pd.read_csv("novel-corona-virus/sars_2003_complete_dataset_clean.csv")
df_sars_03['Date'] = pd.to_datetime(df_sars_03['Date'])
df_sars_03['Province/State'] = None
df_sars_03['Outbreak'] = 'SARS_2003'
df_sars_03.rename({'Cumulative number of case(s)':'Confirmed', 'Number of deaths':'Deaths', 'Number recovered':'Recovered', 'Country':'Country/Region'},axis=1,inplace=True)
templ_cols = ['Outbreak', 'Province/State', 'Country/Region', 'Date', 'Confirmed', 'Deaths', 'Recovered']
df = pd.concat([df_covid_19[templ_cols], df_sars_03[templ_cols]])
df=df_covid_19

In [3]:
df = df.reset_index(drop=True)
df['Confirmed'] = df['Confirmed'].fillna(0)
df['Province/State'] = df['Province/State'].fillna('Others')
df = df.sort_values(['Country/Region','Province/State','Date'])
t = df.groupby(['Outbreak','Country/Region','Province/State']).agg({'Confirmed':'max'})
t = t.loc[t['Confirmed'] >=0]
df = pd.merge(df,t,left_on=['Outbreak','Country/Region','Province/State'], right_index=True)

In [4]:
country_data = pd.read_csv("novel-corona-virus/countries of the world.csv")
country_data['Country'] = country_data['Country'].str.strip()
df.loc[df['Country/Region']=='US','Country/Region'] = 'United States'
df.loc[df['Country/Region']=='Mainland China','Country/Region'] = 'China'
df.loc[df['Country/Region']=='Viet Nam','Country/Region'] = 'Vietnam'
df.loc[df['Country/Region']=='UK','Country/Region'] = 'United Kingdom'
df.loc[df['Country/Region']=='South Korea','Country/Region'] = 'Korea, South'
df.loc[df['Country/Region']=='Taiwan, China','Country/Region'] = 'Taiwan'
df.loc[df['Country/Region']=='Hong Kong SAR, China','Country/Region'] = 'Hong Kong'
df = pd.merge(df, country_data, how='left', left_on=['Country/Region'], right_on=['Country'])
df['Date'] = pd.to_datetime(df['Date'])
data_wthear=pd.read_csv('input/final.txt',sep='\t',error_bad_lines=False)

In [5]:
l=[]
for index,row in data_wthear.iterrows():
    s=row['updated_at'].split(' ')
    l.append(s[0])
data_wthear['updated_at']=l
data_wthear=data_wthear.dropna()
data_wthear['updated_at'] = pd.to_datetime(data_wthear['updated_at'])
df_new = pd.merge(df, data_wthear, how='inner', left_on=['Country/Region','Date','Province/State'], right_on=['Country/Region','updated_at','Province/State'])

In [6]:
df=df_new.copy()
df.loc[df['Region'].isnull(), 'Region'] = 'Others'
df.loc[df['Country'].isnull(), 'Country'] = 'Undefined'
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.week
df = df.fillna(0)
df=df.drop('Other (%)',axis=1)

In [7]:
import locale
from fastnumbers import fast_real
Literacy=[]
Pop,Pho,Agric=[],[],[]
Coa,Clim,Bir,Dea=[],[],[],[]
Net,Inf,Arab,Crop,Ind,Serv=[],[],[],[],[],[]
for index ,row in df.iterrows():
    a=str(row['Literacy (%)']).replace(',','.')
    Literacy.append(fast_real(a))
    #----------
    a=str(row['Pop. Density (per sq. mi.)']).replace(',','.')
    Pop.append(fast_real(a))
    #----------
    a=str(row['Coastline (coast/area ratio)']).replace(',','.')
    Coa.append(fast_real(a))
    #-----------
    a=str(row['Net migration']).replace(',','.')
    Net.append(fast_real(a))
    #-----------
    a=str(row['Infant mortality (per 1000 births)']).replace(',','.')
    Inf.append(fast_real(a))
    #
    #
    a=str(row['Phones (per 1000)']).replace(',','.')
    Pho.append(fast_real(a))
    #
    a=str(row['Arable (%)']).replace(',','.')
    Arab.append(fast_real(a))
    #
    a=str(row['Crops (%)']).replace(',','.')
    Crop.append(fast_real(a))
    #
    a=str(row['Climate']).replace(',','.')
    Clim.append(fast_real(a))
    #
    a=str(row['Birthrate']).replace(',','.')
    Bir.append(fast_real(a))
    #
    a=str(row['Deathrate']).replace(',','.')
    Dea.append(fast_real(a))
    #
    a=str(row['Agriculture']).replace(',','.')
    Agric.append(fast_real(a))
    #
    a=str(row['Industry']).replace(',','.')
    Ind.append(fast_real(a))
    #
    a=str(row['Service']).replace(',','.')
    Serv.append(fast_real(a))
df['Literacy (%)']=Literacy
df['Pop. Density (per sq. mi.)']=Pop
df['Coastline (coast/area ratio)']=Coa
df['Net migration']=Net
df['Infant mortality (per 1000 births)']=Inf
df['Phones (per 1000)']=Pho
df['Arable (%)']=Arab
df['Crops (%)']=Crop
df['Climate']=Clim
df['Birthrate']=Bir
df['Deathrate']=Dea
df['Agriculture']=Agric
df['Industry']=Ind
df['Service']=Serv
df.head()
df['Country/Region'] = df['Country/Region'].astype('category')
df['Outbreak'] = df['Outbreak'].astype('category')
df['precipType'] = df['precipType'].astype('category')
df['Country/Region'] = df['Country/Region'].cat.codes
df['Outbreak'] = df['Outbreak'].cat.codes
df['precipType'] = df['precipType'].cat.codes

In [8]:
def base_model(feature_len=3, after_day=3, input_shape=(8, 1)):
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=False, input_shape=input_shape))
    #model.add(LSTM(units=100, return_sequences=False, input_shape=input_shape))
    # one to many
    model.add(RepeatVector(after_day))
    #model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(50, return_sequences=True))
    model.add(TimeDistributed(Dense(units=feature_len, activation='linear')))
    return model
def normalize_data(data, scaler, feature_len):
    minmaxscaler = scaler.fit(data)
    normalize_data = minmaxscaler.transform(data)
    return normalize_data
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM,GRU
from keras.layers import Dense
from keras import backend as K
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, Activation, TimeDistributed, Dropout, Lambda, RepeatVector, Input, Reshape
from keras.callbacks import ModelCheckpoint
def load_data(data, time_step=2, after_day=1, validate_percent=0.67):
    seq_length = time_step + after_day
    result = []
    for index in range(len(data) - seq_length + 1):
        result.append(data[index: index + seq_length])
    result = np.array(result)
    print('total data: ', result.shape)
    train_size = int(len(result) * validate_percent)
    train = result[:train_size, :]
    validate = result[train_size:, :]
    x_train = train[:, :time_step]
    y_train = train[:, time_step:]
    x_validate = validate[:, :time_step]
    y_validate = validate[:, time_step:]
    return [x_train, y_train, x_validate, y_validate]
def base_model2(feature_len=3, after_day=3, input_shape=(8, 1)):
    model = Sequential()
    model.add(GRU(units=100, return_sequences=False, input_shape=input_shape))
    #model.add(LSTM(units=100, return_sequences=False, input_shape=input_shape))
    # one to many
    model.add(RepeatVector(after_day))
    #model.add(LSTM(200, return_sequences=True))
    model.add(GRU(50, return_sequences=True))
    model.add(TimeDistributed(Dense(units=feature_len, activation='linear')))
    return model
def seq2seq(feature_len=1, after_day=1, input_shape=(8, 1)):
    '''
    Encoder:
    X = Input sequence
    C = LSTM(X); The context vector

    Decoder:
    y(t) = LSTM(s(t-1), y(t-1)); where s is the hidden state of the LSTM(h and c)
    y(0) = LSTM(s0, C); C is the context vector from the encoder.
    '''

    # Encoder
    encoder_inputs = Input(shape=input_shape) # (timesteps, feature)
    encoder = LSTM(units=100, return_state=True,  name='encoder')
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    states = [state_h, state_c]

    # Decoder
    reshapor = Reshape((1, 100), name='reshapor')
    decoder = LSTM(units=100, return_sequences=True, return_state=True, name='decoder')

    # Densor
    #tdensor = TimeDistributed(Dense(units=200, activation='linear', name='time_densor'))
    densor_output = Dense(units=feature_len, activation='linear', name='output')

    inputs = reshapor(encoder_outputs)
    #inputs = tdensor(inputs)
    all_outputs = []



    for _ in range(after_day):
        outputs, h, c = decoder(inputs, initial_state=states)

        #inputs = tdensor(outputs)
        inputs = outputs
        states = [state_h, state_c]

        outputs = densor_output(outputs)
        all_outputs.append(outputs)

    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)
    model = Model(inputs=encoder_inputs, outputs=decoder_outputs)

    return model
 # model complie
from keras.optimizers import SGD,Adam
# root mean squared error (rmse) for regression (only for Keras tensors)
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

Using TensorFlow backend.
/home/rad/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rad/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rad/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rad/anacond

In [9]:
df.keys()

Index(['SNo', 'ObservationDate', 'Province/State', 'Country/Region',
       'Last Update', 'Confirmed_x', 'Deaths', 'Recovered', 'Date', 'Outbreak',
       'Confirmed_y', 'Country', 'Region', 'Population', 'Area (sq. mi.)',
       'Pop. Density (per sq. mi.)', 'Coastline (coast/area ratio)',
       'Net migration', 'Infant mortality (per 1000 births)',
       'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)', 'Arable (%)',
       'Crops (%)', 'Climate', 'Birthrate', 'Deathrate', 'Agriculture',
       'Industry', 'Service', 'id', 'CountryID', 'DateTime', 'Res_status',
       'API', 'Timezone', 'Time', 'Summary', 'icon', 'sunriseTime',
       'sunsetTime', 'moonPhase', 'precipIntensity', 'precipIntensityMax',
       'precipIntensityMaxTime', 'precipProbability', 'precipType',
       'temperatureHigh', 'temperatureHighTime', 'temperatureLow',
       'temperatureLowTime', 'apparentTemperatureHigh',
       'apparentTemperatureHighTime', 'apparentTemperatureLow',
       'apparentTemp

# humidity

In [11]:
col=[ 'Outbreak',  'Country/Region', 'humidity','Confirmed_x','Deaths','Recovered']
newdf=df[col]
newdf.head()

,Outbreak,Country/Region,humidity,Confirmed_x,Deaths,Recovered
0,0,0,0.11,112.0,2.0,4.0
1,0,0,0.11,112.0,2.0,4.0
2,0,0,0.16,112.0,2.0,4.0
3,0,0,0.16,112.0,2.0,4.0
4,0,0,0.28,112.0,2.0,4.0


# RUN MODELs

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 8s 212us/step - loss: 0.0657 - acc: 0.4096 - rmse: 0.2204 - mse: 0.0657 - r_square: 0.2902 - val_loss: 0.0192 - val_acc: 0.5578 - val_rmse: 0.1227 - val_mse: 0.0192 - val_r_square: 0.8102
Epoch 2/100
35456/35456 [==============================] - 4s 108us/step - loss: 0.0140 - acc: 0.4769 - rmse: 0.1055 - mse: 0.0140 - r_square: 0.8439 - val_loss: 0.0124 - val_acc: 0.6247 - val_rmse: 0.0994 - val_mse: 0.0124 - val_r_square: 0.8773
Epoch 3/100
35456/35456 [==============================] - 4s 113us/s

35456/35456 [==============================] - 3s 94us/step - loss: 0.0037 - acc: 0.7371 - rmse: 0.0509 - mse: 0.0037 - r_square: 0.9550 - val_loss: 0.0070 - val_acc: 0.7070 - val_rmse: 0.0690 - val_mse: 0.0070 - val_r_square: 0.9315
Epoch 64/100
35456/35456 [==============================] - 3s 94us/step - loss: 0.0038 - acc: 0.7391 - rmse: 0.0504 - mse: 0.0038 - r_square: 0.9547 - val_loss: 0.0071 - val_acc: 0.6424 - val_rmse: 0.0692 - val_mse: 0.0071 - val_r_square: 0.9289
Epoch 65/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0039 - acc: 0.7366 - rmse: 0.0514 - mse: 0.0039 - r_square: 0.9532 - val_loss: 0.0067 - val_acc: 0.6913 - val_rmse: 0.0683 - val_mse: 0.0067 - val_r_square: 0.9321
Epoch 66/100
35456/35456 [==============================] - 3s 94us/step - loss: 0.0039 - acc: 0.7331 - rmse: 0.0510 - mse: 0.0039 - r_square: 0.9543 - val_loss: 0.0072 - val_acc: 0.6453 - val_rmse: 0.0679 - val_mse: 0.0072 - val_r_square: 0.9301
Epoch 67/100
35456/35456 

35456/35456 [==============================] - 5s 147us/step - loss: 0.0032 - acc: 0.7264 - rmse: 0.0460 - mse: 0.0032 - r_square: 0.9598 - val_loss: 0.0071 - val_acc: 0.6008 - val_rmse: 0.0637 - val_mse: 0.0071 - val_r_square: 0.9337
Epoch 29/100
35456/35456 [==============================] - 6s 155us/step - loss: 0.0031 - acc: 0.7334 - rmse: 0.0452 - mse: 0.0031 - r_square: 0.9605 - val_loss: 0.0072 - val_acc: 0.6003 - val_rmse: 0.0651 - val_mse: 0.0072 - val_r_square: 0.9317
Epoch 30/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0031 - acc: 0.7363 - rmse: 0.0447 - mse: 0.0031 - r_square: 0.9609 - val_loss: 0.0076 - val_acc: 0.5998 - val_rmse: 0.0680 - val_mse: 0.0076 - val_r_square: 0.9278
Epoch 31/100
35456/35456 [==============================] - 5s 145us/step - loss: 0.0031 - acc: 0.7382 - rmse: 0.0447 - mse: 0.0031 - r_square: 0.9609 - val_loss: 0.0081 - val_acc: 0.6029 - val_rmse: 0.0721 - val_mse: 0.0081 - val_r_square: 0.9220
Epoch 32/100
35456/35

35456/35456 [==============================] - 5s 151us/step - loss: 0.0030 - acc: 0.7448 - rmse: 0.0433 - mse: 0.0030 - r_square: 0.9626 - val_loss: 0.0083 - val_acc: 0.6794 - val_rmse: 0.0707 - val_mse: 0.0083 - val_r_square: 0.9229
Epoch 95/100
35456/35456 [==============================] - 5s 151us/step - loss: 0.0029 - acc: 0.7431 - rmse: 0.0431 - mse: 0.0029 - r_square: 0.9629 - val_loss: 0.0082 - val_acc: 0.6801 - val_rmse: 0.0710 - val_mse: 0.0082 - val_r_square: 0.9240
Epoch 96/100
35456/35456 [==============================] - 5s 148us/step - loss: 0.0030 - acc: 0.7441 - rmse: 0.0437 - mse: 0.0030 - r_square: 0.9625 - val_loss: 0.0085 - val_acc: 0.6887 - val_rmse: 0.0719 - val_mse: 0.0085 - val_r_square: 0.9215
Epoch 97/100
35456/35456 [==============================] - 6s 164us/step - loss: 0.0031 - acc: 0.7303 - rmse: 0.0451 - mse: 0.0031 - r_square: 0.9615 - val_loss: 0.0084 - val_acc: 0.6878 - val_rmse: 0.0719 - val_mse: 0.0084 - val_r_square: 0.9230
Epoch 98/100
35456/35

35456/35456 [==============================] - 5s 132us/step - loss: 0.0039 - acc: 0.7787 - mse: 0.0039 - rmse: 0.0504 - r_square: 0.9492 - val_loss: 0.0165 - val_acc: 0.6385 - val_mse: 0.0165 - val_rmse: 0.1166 - val_r_square: 0.8346
Epoch 60/100
35456/35456 [==============================] - 5s 132us/step - loss: 0.0037 - acc: 0.7841 - mse: 0.0037 - rmse: 0.0493 - r_square: 0.9527 - val_loss: 0.0164 - val_acc: 0.6387 - val_mse: 0.0164 - val_rmse: 0.1163 - val_r_square: 0.8348
Epoch 61/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0037 - acc: 0.7842 - mse: 0.0037 - rmse: 0.0497 - r_square: 0.9525 - val_loss: 0.0157 - val_acc: 0.6403 - val_mse: 0.0157 - val_rmse: 0.1132 - val_r_square: 0.8402
Epoch 62/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0036 - acc: 0.7855 - mse: 0.0036 - rmse: 0.0490 - r_square: 0.9539 - val_loss: 0.0161 - val_acc: 0.6385 - val_mse: 0.0161 - val_rmse: 0.1149 - val_r_square: 0.8380
Epoch 63/100
35456/35

# precipIntensity

In [21]:
col=[ 'Outbreak',  'Country/Region', 'precipIntensity','Confirmed_x','Deaths','Recovered']
newdf=df[col]

In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 7s 203us/step - loss: 0.0423 - acc: 0.7992 - rmse: 0.1665 - mse: 0.0423 - r_square: 0.5202 - val_loss: 0.0094 - val_acc: 0.9719 - val_rmse: 0.0757 - val_mse: 0.0094 - val_r_square: 0.9001
Epoch 2/100
35456/35456 [==============================] - 4s 105us/step - loss: 0.0053 - acc: 0.8516 - rmse: 0.0604 - mse: 0.0053 - r_square: 0.9133 - val_loss: 0.0061 - val_acc: 0.9719 - val_rmse: 0.0606 - val_mse: 0.0061 - val_r_square: 0.9407
Epoch 3/100
35456/35456 [==============================] - 4s 104us/step - loss: 0.0042 - acc: 0.8639 - rmse: 0.0530 - mse: 0.0042 - r_square: 0.9223 - val_loss: 0.0049 - val_acc: 0.9993 - 

Epoch 33/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0011 - acc: 0.9410 - rmse: 0.0255 - mse: 0.0011 - r_square: 0.9819 - val_loss: 0.0020 - val_acc: 0.9993 - val_rmse: 0.0320 - val_mse: 0.0020 - val_r_square: 0.9822
Epoch 34/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0011 - acc: 0.9445 - rmse: 0.0253 - mse: 0.0011 - r_square: 0.9821 - val_loss: 0.0021 - val_acc: 0.9993 - val_rmse: 0.0316 - val_mse: 0.0021 - val_r_square: 0.9823
Epoch 35/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0011 - acc: 0.9482 - rmse: 0.0253 - mse: 0.0011 - r_square: 0.9820 - val_loss: 0.0021 - val_acc: 0.9993 - val_rmse: 0.0314 - val_mse: 0.0021 - val_r_square: 0.9823
Epoch 36/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0011 - acc: 0.9464 - rmse: 0.0254 - mse: 0.0011 - r_square: 0.9817 - val_loss: 0.0021 - val_acc: 0.9993 - val_rmse: 0.0318 - val_mse: 0.0021 - val_r_square: 0.9819
Epoch 37/100

35456/35456 [==============================] - 3s 99us/step - loss: 6.9203e-04 - acc: 0.9448 - rmse: 0.0184 - mse: 6.9203e-04 - r_square: 0.9846 - val_loss: 0.0028 - val_acc: 0.9914 - val_rmse: 0.0389 - val_mse: 0.0028 - val_r_square: 0.9736
Epoch 99/100
35456/35456 [==============================] - 3s 95us/step - loss: 7.3918e-04 - acc: 0.9321 - rmse: 0.0186 - mse: 7.3918e-04 - r_square: 0.9846 - val_loss: 0.0029 - val_acc: 0.9993 - val_rmse: 0.0394 - val_mse: 0.0029 - val_r_square: 0.9727
Epoch 100/100
35456/35456 [==============================] - 4s 105us/step - loss: 7.7399e-04 - acc: 0.9386 - rmse: 0.0195 - mse: 7.7399e-04 - r_square: 0.9839 - val_loss: 0.0030 - val_acc: 0.9193 - val_rmse: 0.0413 - val_mse: 0.0030 - val_r_square: 0.9705
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [===========

35456/35456 [==============================] - 5s 144us/step - loss: 7.6208e-04 - acc: 0.9504 - rmse: 0.0198 - mse: 7.6208e-04 - r_square: 0.9745 - val_loss: 0.0018 - val_acc: 0.8899 - val_rmse: 0.0296 - val_mse: 0.0018 - val_r_square: 0.9839
Epoch 63/100
35456/35456 [==============================] - 5s 142us/step - loss: 6.8389e-04 - acc: 0.9518 - rmse: 0.0190 - mse: 6.8389e-04 - r_square: 0.9848 - val_loss: 0.0020 - val_acc: 0.8898 - val_rmse: 0.0350 - val_mse: 0.0020 - val_r_square: 0.9793
Epoch 64/100
35456/35456 [==============================] - 5s 142us/step - loss: 5.9540e-04 - acc: 0.9491 - rmse: 0.0165 - mse: 5.9540e-04 - r_square: 0.9830 - val_loss: 0.0016 - val_acc: 0.8897 - val_rmse: 0.0274 - val_mse: 0.0016 - val_r_square: 0.9857
Epoch 65/100
35456/35456 [==============================] - 5s 146us/step - loss: 5.6956e-04 - acc: 0.9511 - rmse: 0.0160 - mse: 5.6956e-04 - r_square: 0.9844 - val_loss: 0.0019 - val_acc: 0.8820 - val_rmse: 0.0327 - val_mse: 0.0019 - val_r_squa

35456/35456 [==============================] - 5s 140us/step - loss: 0.0016 - acc: 0.9774 - mse: 0.0016 - rmse: 0.0263 - r_square: 0.9724 - val_loss: 0.0094 - val_acc: 0.9993 - val_mse: 0.0094 - val_rmse: 0.0837 - val_r_square: 0.8980
Epoch 27/100
35456/35456 [==============================] - 5s 133us/step - loss: 0.0018 - acc: 0.9752 - mse: 0.0018 - rmse: 0.0285 - r_square: 0.9685 - val_loss: 0.0093 - val_acc: 0.9993 - val_mse: 0.0093 - val_rmse: 0.0841 - val_r_square: 0.8959
Epoch 28/100
35456/35456 [==============================] - 5s 134us/step - loss: 0.0017 - acc: 0.9738 - mse: 0.0017 - rmse: 0.0287 - r_square: 0.9682 - val_loss: 0.0093 - val_acc: 0.9993 - val_mse: 0.0093 - val_rmse: 0.0841 - val_r_square: 0.8954
Epoch 29/100
35456/35456 [==============================] - 5s 140us/step - loss: 0.0016 - acc: 0.9770 - mse: 0.0016 - rmse: 0.0263 - r_square: 0.9664 - val_loss: 0.0092 - val_acc: 0.9993 - val_mse: 0.0092 - val_rmse: 0.0813 - val_r_square: 0.9048
Epoch 30/100
35456/35

Epoch 59/100
35456/35456 [==============================] - 5s 131us/step - loss: 7.3611e-04 - acc: 0.9788 - mse: 7.3611e-04 - rmse: 0.0187 - r_square: 0.9789 - val_loss: 0.0044 - val_acc: 0.9993 - val_mse: 0.0044 - val_rmse: 0.0423 - val_r_square: 0.9661
Epoch 60/100
35456/35456 [==============================] - 5s 130us/step - loss: 7.4728e-04 - acc: 0.9739 - mse: 7.4728e-04 - rmse: 0.0184 - r_square: 0.9823 - val_loss: 0.0062 - val_acc: 0.9993 - val_mse: 0.0062 - val_rmse: 0.0626 - val_r_square: 0.9423
Epoch 61/100
35456/35456 [==============================] - 5s 130us/step - loss: 9.1379e-04 - acc: 0.9772 - mse: 9.1379e-04 - rmse: 0.0209 - r_square: 0.9810 - val_loss: 0.0045 - val_acc: 0.9994 - val_mse: 0.0045 - val_rmse: 0.0435 - val_r_square: 0.9643
Epoch 62/100
35456/35456 [==============================] - 5s 130us/step - loss: 7.4468e-04 - acc: 0.9801 - mse: 7.4468e-04 - rmse: 0.0185 - r_square: 0.9747 - val_loss: 0.0044 - val_acc: 0.9994 - val_mse: 0.0044 - val_rmse: 0.0446

# precipIntensityMax

In [25]:
col=[ 'Outbreak',  'Country/Region', 'precipIntensityMax','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 9s 241us/step - loss: 0.0423 - acc: 0.8067 - rmse: 0.1655 - mse: 0.0423 - r_square: 0.5138 - val_loss: 0.0097 - val_acc: 0.9445 - val_rmse: 0.0790 - val_mse: 0.0097 - val_r_square: 0.8945
Epoch 2/100
35456/35456 [==============================] - 4s 106us/step - loss: 0.0050 - acc: 0.8727 - rmse: 0.0592 - mse: 0.0050 - r_square: 0.9148 - val_loss: 0.0058 - val_acc: 0.9719 - val_rmse: 0.0612 - val_mse: 0.0058 - val_r_square: 0.9424
Epoch 3/100
35456/35456 [==============================] - 4s 115us/step - loss: 0.0039 - acc: 0.8698 - rmse: 0.0514 - mse: 0.0039 - r_square: 0.9246 - val_loss: 0.0050 - val_acc: 0.9993 - 

35456/35456 [==============================] - 3s 95us/step - loss: 0.0012 - acc: 0.9494 - rmse: 0.0232 - mse: 0.0012 - r_square: 0.9783 - val_loss: 0.0030 - val_acc: 0.9993 - val_rmse: 0.0438 - val_mse: 0.0030 - val_r_square: 0.9670
Epoch 66/100
35456/35456 [==============================] - 3s 98us/step - loss: 0.0012 - acc: 0.9479 - rmse: 0.0234 - mse: 0.0012 - r_square: 0.9790 - val_loss: 0.0033 - val_acc: 0.9993 - val_rmse: 0.0479 - val_mse: 0.0033 - val_r_square: 0.9603
Epoch 67/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0012 - acc: 0.9496 - rmse: 0.0231 - mse: 0.0012 - r_square: 0.9787 - val_loss: 0.0029 - val_acc: 0.9993 - val_rmse: 0.0432 - val_mse: 0.0029 - val_r_square: 0.9677
Epoch 68/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0012 - acc: 0.9465 - rmse: 0.0231 - mse: 0.0012 - r_square: 0.9790 - val_loss: 0.0032 - val_acc: 0.9993 - val_rmse: 0.0470 - val_mse: 0.0032 - val_r_square: 0.9617
Epoch 69/100
35456/35456 

Epoch 98/100
35456/35456 [==============================] - 3s 97us/step - loss: 7.2839e-04 - acc: 0.9576 - rmse: 0.0178 - mse: 7.2839e-04 - r_square: 0.9857 - val_loss: 0.0029 - val_acc: 0.9719 - val_rmse: 0.0401 - val_mse: 0.0029 - val_r_square: 0.9726
Epoch 99/100
35456/35456 [==============================] - 3s 97us/step - loss: 8.6816e-04 - acc: 0.9524 - rmse: 0.0208 - mse: 8.6816e-04 - r_square: 0.9843 - val_loss: 0.0034 - val_acc: 0.9719 - val_rmse: 0.0472 - val_mse: 0.0034 - val_r_square: 0.9632
Epoch 100/100
35456/35456 [==============================] - 3s 97us/step - loss: 0.0011 - acc: 0.9345 - rmse: 0.0236 - mse: 0.0011 - r_square: 0.9820 - val_loss: 0.0034 - val_acc: 0.9720 - val_rmse: 0.0463 - val_mse: 0.0034 - val_r_square: 0.9641
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [=======

Epoch 30/100
35456/35456 [==============================] - 6s 161us/step - loss: 0.0010 - acc: 0.9315 - rmse: 0.0238 - mse: 0.0010 - r_square: 0.9800 - val_loss: 0.0024 - val_acc: 0.8902 - val_rmse: 0.0399 - val_mse: 0.0024 - val_r_square: 0.9737
Epoch 31/100
35456/35456 [==============================] - 6s 174us/step - loss: 0.0011 - acc: 0.9296 - rmse: 0.0252 - mse: 0.0011 - r_square: 0.9790 - val_loss: 0.0030 - val_acc: 0.8898 - val_rmse: 0.0468 - val_mse: 0.0030 - val_r_square: 0.9641
Epoch 32/100
35456/35456 [==============================] - 7s 186us/step - loss: 0.0012 - acc: 0.9261 - rmse: 0.0259 - mse: 0.0012 - r_square: 0.9780 - val_loss: 0.0034 - val_acc: 0.8897 - val_rmse: 0.0512 - val_mse: 0.0034 - val_r_square: 0.9563
Epoch 33/100
35456/35456 [==============================] - 6s 170us/step - loss: 0.0012 - acc: 0.9378 - rmse: 0.0259 - mse: 0.0012 - r_square: 0.9781 - val_loss: 0.0040 - val_acc: 0.8897 - val_rmse: 0.0557 - val_mse: 0.0040 - val_r_square: 0.9473
Epoch 34

Epoch 63/100
35456/35456 [==============================] - 5s 153us/step - loss: 0.0011 - acc: 0.9481 - rmse: 0.0242 - mse: 0.0011 - r_square: 0.9813 - val_loss: 0.0024 - val_acc: 0.8901 - val_rmse: 0.0350 - val_mse: 0.0024 - val_r_square: 0.9791
Epoch 64/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0011 - acc: 0.9494 - rmse: 0.0246 - mse: 0.0011 - r_square: 0.9788 - val_loss: 0.0024 - val_acc: 0.8914 - val_rmse: 0.0342 - val_mse: 0.0024 - val_r_square: 0.9793
Epoch 65/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0010 - acc: 0.9512 - rmse: 0.0235 - mse: 0.0010 - r_square: 0.9821 - val_loss: 0.0024 - val_acc: 0.8901 - val_rmse: 0.0341 - val_mse: 0.0024 - val_r_square: 0.9797
Epoch 66/100
35456/35456 [==============================] - 5s 149us/step - loss: 0.0010 - acc: 0.9503 - rmse: 0.0233 - mse: 0.0010 - r_square: 0.9800 - val_loss: 0.0021 - val_acc: 0.8919 - val_rmse: 0.0299 - val_mse: 0.0021 - val_r_square: 0.9823
Epoch 67

35456/35456 [==============================] - 5s 134us/step - loss: 0.0019 - acc: 0.9734 - mse: 0.0019 - rmse: 0.0302 - r_square: 0.9549 - val_loss: 0.0087 - val_acc: 0.9993 - val_mse: 0.0087 - val_rmse: 0.0781 - val_r_square: 0.9096
Epoch 28/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0018 - acc: 0.9751 - mse: 0.0018 - rmse: 0.0287 - r_square: 0.9601 - val_loss: 0.0092 - val_acc: 0.9994 - val_mse: 0.0092 - val_rmse: 0.0821 - val_r_square: 0.8950
Epoch 29/100
35456/35456 [==============================] - 5s 135us/step - loss: 0.0020 - acc: 0.9759 - mse: 0.0020 - rmse: 0.0302 - r_square: 0.9320 - val_loss: 0.0091 - val_acc: 0.9994 - val_mse: 0.0091 - val_rmse: 0.0820 - val_r_square: 0.9012
Epoch 30/100
35456/35456 [==============================] - 5s 135us/step - loss: 0.0021 - acc: 0.9702 - mse: 0.0021 - rmse: 0.0323 - r_square: 0.9367 - val_loss: 0.0097 - val_acc: 0.9993 - val_mse: 0.0097 - val_rmse: 0.0858 - val_r_square: 0.8902
Epoch 31/100
35456/35

35456/35456 [==============================] - 5s 134us/step - loss: 0.0019 - acc: 0.9749 - mse: 0.0019 - rmse: 0.0280 - r_square: 0.9578 - val_loss: 0.0101 - val_acc: 0.9994 - val_mse: 0.0101 - val_rmse: 0.0881 - val_r_square: 0.8764
Epoch 94/100
35456/35456 [==============================] - 5s 135us/step - loss: 0.0019 - acc: 0.9775 - mse: 0.0019 - rmse: 0.0282 - r_square: 0.9651 - val_loss: 0.0101 - val_acc: 0.9994 - val_mse: 0.0101 - val_rmse: 0.0885 - val_r_square: 0.8755
Epoch 95/100
35456/35456 [==============================] - 5s 139us/step - loss: 0.0020 - acc: 0.9765 - mse: 0.0020 - rmse: 0.0293 - r_square: 0.9642 - val_loss: 0.0105 - val_acc: 0.9994 - val_mse: 0.0105 - val_rmse: 0.0909 - val_r_square: 0.8681
Epoch 96/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0020 - acc: 0.9766 - mse: 0.0020 - rmse: 0.0293 - r_square: 0.9641 - val_loss: 0.0100 - val_acc: 0.9994 - val_mse: 0.0100 - val_rmse: 0.0882 - val_r_square: 0.8762
Epoch 97/100
35456/35

# temperatureHigh

In [27]:
col=[ 'Outbreak',  'Country/Region', 'temperatureHigh','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 9s 242us/step - loss: 0.0553 - acc: 0.6861 - rmse: 0.2007 - mse: 0.0553 - r_square: 0.2884 - val_loss: 0.0156 - val_acc: 0.8362 - val_rmse: 0.1078 - val_mse: 0.0156 - val_r_square: 0.8338
Epoch 2/100
35456/35456 [==============================] - 4s 110us/step - loss: 0.0107 - acc: 0.7526 - rmse: 0.0920 - mse: 0.0107 - r_square: 0.8574 - val_loss: 0.0079 - val_acc: 0.9182 - val_rmse: 0.0763 - val_mse: 0.0079 - val_r_square: 0.9174
Epoch 3/100
35456/35456 [==============================] - 5s 132us/step - loss: 0.0063 - acc: 0.8069 - rmse: 0.0691 - mse: 0.0063 - r_square: 0.9095 - val_loss: 0.0071 - val_acc: 0.9181 - 

Epoch 33/100
35456/35456 [==============================] - 4s 105us/step - loss: 0.0014 - acc: 0.8511 - rmse: 0.0313 - mse: 0.0014 - r_square: 0.9795 - val_loss: 0.0030 - val_acc: 0.8492 - val_rmse: 0.0422 - val_mse: 0.0030 - val_r_square: 0.9703
Epoch 34/100
35456/35456 [==============================] - 4s 108us/step - loss: 0.0015 - acc: 0.8558 - rmse: 0.0323 - mse: 0.0015 - r_square: 0.9784 - val_loss: 0.0037 - val_acc: 0.8485 - val_rmse: 0.0471 - val_mse: 0.0037 - val_r_square: 0.9637
Epoch 35/100
35456/35456 [==============================] - 4s 108us/step - loss: 0.0018 - acc: 0.8321 - rmse: 0.0354 - mse: 0.0018 - r_square: 0.9742 - val_loss: 0.0038 - val_acc: 0.8432 - val_rmse: 0.0477 - val_mse: 0.0038 - val_r_square: 0.9614
Epoch 36/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0019 - acc: 0.8299 - rmse: 0.0362 - mse: 0.0019 - r_square: 0.9736 - val_loss: 0.0036 - val_acc: 0.8371 - val_rmse: 0.0447 - val_mse: 0.0036 - val_r_square: 0.9649
Epoch 37/

35456/35456 [==============================] - 3s 99us/step - loss: 0.0011 - acc: 0.9169 - rmse: 0.0275 - mse: 0.0011 - r_square: 0.9831 - val_loss: 0.0039 - val_acc: 0.8455 - val_rmse: 0.0460 - val_mse: 0.0039 - val_r_square: 0.9633
Epoch 100/100
35456/35456 [==============================] - 3s 97us/step - loss: 0.0012 - acc: 0.9368 - rmse: 0.0277 - mse: 0.0012 - r_square: 0.9829 - val_loss: 0.0043 - val_acc: 0.8493 - val_rmse: 0.0469 - val_mse: 0.0043 - val_r_square: 0.9617
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 12s 344us/step - loss: 0.0348 - acc: 0.7772 - rmse: 0.1402 - mse: 0.0348 - r_square: 0.5671 - val_loss: 0.0058 - val_acc: 0.8694 - val_rmse: 0.0650 - val_mse: 0.0058 - val_r_square: 0.9382
Epoch 2/100
35456/35456 [==============================] - 5

35456/35456 [==============================] - 5s 138us/step - loss: 0.0013 - acc: 0.8972 - rmse: 0.0292 - mse: 0.0013 - r_square: 0.9818 - val_loss: 0.0034 - val_acc: 0.8415 - val_rmse: 0.0444 - val_mse: 0.0034 - val_r_square: 0.9667
Epoch 65/100
35456/35456 [==============================] - 5s 138us/step - loss: 0.0012 - acc: 0.9011 - rmse: 0.0290 - mse: 0.0012 - r_square: 0.9817 - val_loss: 0.0029 - val_acc: 0.8491 - val_rmse: 0.0401 - val_mse: 0.0029 - val_r_square: 0.9719
Epoch 66/100
35456/35456 [==============================] - 5s 138us/step - loss: 0.0011 - acc: 0.9050 - rmse: 0.0270 - mse: 0.0011 - r_square: 0.9838 - val_loss: 0.0031 - val_acc: 0.8459 - val_rmse: 0.0407 - val_mse: 0.0031 - val_r_square: 0.9703
Epoch 67/100
35456/35456 [==============================] - 5s 138us/step - loss: 0.0012 - acc: 0.9049 - rmse: 0.0279 - mse: 0.0012 - r_square: 0.9827 - val_loss: 0.0031 - val_acc: 0.8267 - val_rmse: 0.0418 - val_mse: 0.0031 - val_r_square: 0.9691
Epoch 68/100
35456/35

Epoch 97/100
35456/35456 [==============================] - 5s 139us/step - loss: 0.0010 - acc: 0.9117 - rmse: 0.0250 - mse: 0.0010 - r_square: 0.9850 - val_loss: 0.0027 - val_acc: 0.8505 - val_rmse: 0.0373 - val_mse: 0.0027 - val_r_square: 0.9739
Epoch 98/100
35456/35456 [==============================] - 5s 138us/step - loss: 9.9866e-04 - acc: 0.9136 - rmse: 0.0249 - mse: 9.9866e-04 - r_square: 0.9855 - val_loss: 0.0030 - val_acc: 0.8465 - val_rmse: 0.0404 - val_mse: 0.0030 - val_r_square: 0.9703
Epoch 99/100
35456/35456 [==============================] - 5s 138us/step - loss: 0.0010 - acc: 0.9070 - rmse: 0.0258 - mse: 0.0010 - r_square: 0.9846 - val_loss: 0.0030 - val_acc: 0.8501 - val_rmse: 0.0402 - val_mse: 0.0030 - val_r_square: 0.9706
Epoch 100/100
35456/35456 [==============================] - 5s 138us/step - loss: 0.0011 - acc: 0.9039 - rmse: 0.0260 - mse: 0.0011 - r_square: 0.9846 - val_loss: 0.0033 - val_acc: 0.8468 - val_rmse: 0.0427 - val_mse: 0.0033 - val_r_square: 0.9677

35456/35456 [==============================] - 5s 140us/step - loss: 0.0032 - acc: 0.8809 - mse: 0.0032 - rmse: 0.0450 - r_square: 0.9516 - val_loss: 0.0100 - val_acc: 0.9455 - val_mse: 0.0100 - val_rmse: 0.0848 - val_r_square: 0.8998
Epoch 63/100
35456/35456 [==============================] - 5s 140us/step - loss: 0.0033 - acc: 0.8966 - mse: 0.0033 - rmse: 0.0451 - r_square: 0.9526 - val_loss: 0.0099 - val_acc: 0.9455 - val_mse: 0.0099 - val_rmse: 0.0849 - val_r_square: 0.9004
Epoch 64/100
35456/35456 [==============================] - 5s 140us/step - loss: 0.0030 - acc: 0.8965 - mse: 0.0030 - rmse: 0.0431 - r_square: 0.9583 - val_loss: 0.0085 - val_acc: 0.9455 - val_mse: 0.0085 - val_rmse: 0.0760 - val_r_square: 0.9164
Epoch 65/100
35456/35456 [==============================] - 5s 140us/step - loss: 0.0025 - acc: 0.9127 - mse: 0.0025 - rmse: 0.0400 - r_square: 0.9629 - val_loss: 0.0088 - val_acc: 0.9455 - val_mse: 0.0088 - val_rmse: 0.0775 - val_r_square: 0.9141
Epoch 66/100
35456/35

# temperatureLow

In [30]:
col=[ 'Outbreak',  'Country/Region', 'temperatureLow','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 10s 282us/step - loss: 0.0529 - acc: 0.5573 - rmse: 0.1957 - mse: 0.0529 - r_square: 0.3407 - val_loss: 0.0135 - val_acc: 0.8243 - val_rmse: 0.0987 - val_mse: 0.0135 - val_r_square: 0.8590
Epoch 2/100
35456/35456 [==============================] - 4s 111us/step - loss: 0.0113 - acc: 0.6617 - rmse: 0.0932 - mse: 0.0113 - r_square: 0.8544 - val_loss: 0.0079 - val_acc: 0.9064 - val_rmse: 0.0771 - val_mse: 0.0079 - val_r_square: 0.9175
Epoch 3/100
35456/35456 [==============================] - 4s 115us/step - loss: 0.0077 - acc: 0.6785 - rmse: 0.0754 - mse: 0.0077 - r_square: 0.8962 - val_loss: 0.0080 - val_acc: 0.9063 -

Epoch 33/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0013 - acc: 0.8501 - rmse: 0.0292 - mse: 0.0013 - r_square: 0.9823 - val_loss: 0.0031 - val_acc: 0.8295 - val_rmse: 0.0414 - val_mse: 0.0031 - val_r_square: 0.9718
Epoch 34/100
35456/35456 [==============================] - 3s 97us/step - loss: 0.0013 - acc: 0.8281 - rmse: 0.0303 - mse: 0.0013 - r_square: 0.9810 - val_loss: 0.0036 - val_acc: 0.8151 - val_rmse: 0.0474 - val_mse: 0.0036 - val_r_square: 0.9643
Epoch 35/100
35456/35456 [==============================] - 3s 98us/step - loss: 0.0019 - acc: 0.8255 - rmse: 0.0362 - mse: 0.0019 - r_square: 0.9743 - val_loss: 0.0065 - val_acc: 0.8280 - val_rmse: 0.0673 - val_mse: 0.0065 - val_r_square: 0.9339
Epoch 36/100
35456/35456 [==============================] - 3s 98us/step - loss: 0.0047 - acc: 0.7945 - rmse: 0.0566 - mse: 0.0047 - r_square: 0.9364 - val_loss: 0.0151 - val_acc: 0.8225 - val_rmse: 0.1006 - val_mse: 0.0151 - val_r_square: 0.8508
Epoch 37/100

35456/35456 [==============================] - 3s 98us/step - loss: 0.0014 - acc: 0.8771 - rmse: 0.0296 - mse: 0.0014 - r_square: 0.9815 - val_loss: 0.0044 - val_acc: 0.8563 - val_rmse: 0.0543 - val_mse: 0.0044 - val_r_square: 0.9537
Epoch 100/100
35456/35456 [==============================] - 4s 108us/step - loss: 0.0015 - acc: 0.8763 - rmse: 0.0306 - mse: 0.0015 - r_square: 0.9805 - val_loss: 0.0042 - val_acc: 0.8246 - val_rmse: 0.0502 - val_mse: 0.0042 - val_r_square: 0.9597
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 13s 356us/step - loss: 0.0387 - acc: 0.6062 - rmse: 0.1571 - mse: 0.0387 - r_square: 0.5374 - val_loss: 0.0071 - val_acc: 0.9338 - val_rmse: 0.0733 - val_mse: 0.0071 - val_r_square: 0.9285
Epoch 2/100
35456/35456 [==============================] - 

35456/35456 [==============================] - 5s 136us/step - loss: 0.0011 - acc: 0.8719 - rmse: 0.0268 - mse: 0.0011 - r_square: 0.9835 - val_loss: 0.0030 - val_acc: 0.8287 - val_rmse: 0.0399 - val_mse: 0.0030 - val_r_square: 0.9721
Epoch 65/100
35456/35456 [==============================] - 5s 138us/step - loss: 0.0010 - acc: 0.8851 - rmse: 0.0254 - mse: 0.0010 - r_square: 0.9854 - val_loss: 0.0032 - val_acc: 0.8196 - val_rmse: 0.0384 - val_mse: 0.0032 - val_r_square: 0.9719
Epoch 66/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0011 - acc: 0.8782 - rmse: 0.0264 - mse: 0.0011 - r_square: 0.9838 - val_loss: 0.0029 - val_acc: 0.8286 - val_rmse: 0.0378 - val_mse: 0.0029 - val_r_square: 0.9738
Epoch 67/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0010 - acc: 0.9031 - rmse: 0.0255 - mse: 0.0010 - r_square: 0.9853 - val_loss: 0.0033 - val_acc: 0.8185 - val_rmse: 0.0393 - val_mse: 0.0033 - val_r_square: 0.9713
Epoch 68/100
35456/35

Epoch 97/100
35456/35456 [==============================] - 5s 137us/step - loss: 9.6542e-04 - acc: 0.8929 - rmse: 0.0245 - mse: 9.6542e-04 - r_square: 0.9864 - val_loss: 0.0036 - val_acc: 0.8184 - val_rmse: 0.0378 - val_mse: 0.0036 - val_r_square: 0.9702
Epoch 98/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0010 - acc: 0.8773 - rmse: 0.0256 - mse: 0.0010 - r_square: 0.9850 - val_loss: 0.0031 - val_acc: 0.8210 - val_rmse: 0.0344 - val_mse: 0.0031 - val_r_square: 0.9741
Epoch 99/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0010 - acc: 0.8882 - rmse: 0.0253 - mse: 0.0010 - r_square: 0.9858 - val_loss: 0.0038 - val_acc: 0.8186 - val_rmse: 0.0377 - val_mse: 0.0038 - val_r_square: 0.9695
Epoch 100/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0011 - acc: 0.8684 - rmse: 0.0266 - mse: 0.0011 - r_square: 0.9842 - val_loss: 0.0036 - val_acc: 0.8182 - val_rmse: 0.0402 - val_mse: 0.0036 - val_r_square: 0.9687

35456/35456 [==============================] - 5s 140us/step - loss: 0.0027 - acc: 0.8832 - mse: 0.0027 - rmse: 0.0399 - r_square: 0.9629 - val_loss: 0.0107 - val_acc: 0.9339 - val_mse: 0.0107 - val_rmse: 0.0901 - val_r_square: 0.8853
Epoch 63/100
35456/35456 [==============================] - 5s 141us/step - loss: 0.0028 - acc: 0.8963 - mse: 0.0028 - rmse: 0.0401 - r_square: 0.9625 - val_loss: 0.0106 - val_acc: 0.9337 - val_mse: 0.0106 - val_rmse: 0.0895 - val_r_square: 0.8885
Epoch 64/100
35456/35456 [==============================] - 5s 140us/step - loss: 0.0025 - acc: 0.8935 - mse: 0.0025 - rmse: 0.0391 - r_square: 0.9654 - val_loss: 0.0104 - val_acc: 0.9339 - val_mse: 0.0104 - val_rmse: 0.0889 - val_r_square: 0.8894
Epoch 65/100
35456/35456 [==============================] - 5s 141us/step - loss: 0.0025 - acc: 0.8910 - mse: 0.0025 - rmse: 0.0389 - r_square: 0.9657 - val_loss: 0.0109 - val_acc: 0.9339 - val_mse: 0.0109 - val_rmse: 0.0917 - val_r_square: 0.8836
Epoch 66/100
35456/35

# apparentTemperatureHigh

In [33]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'apparentTemperatureHigh','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 11s 303us/step - loss: 0.0561 - acc: 0.5272 - rmse: 0.2012 - mse: 0.0561 - r_square: 0.3109 - val_loss: 0.0169 - val_acc: 0.7443 - val_rmse: 0.1158 - val_mse: 0.0169 - val_r_square: 0.8247
Epoch 2/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0112 - acc: 0.7146 - rmse: 0.0935 - mse: 0.0112 - r_square: 0.8594 - val_loss: 0.0087 - val_acc: 0.7992 - val_rmse: 0.0811 - val_mse: 0.0087 - val_r_square: 0.9130
Epoch 3/100
35456/35456 [==============================] - 4s 106us/step - loss: 0.0065 - acc: 0.6861 - rmse: 0.0696 - mse: 0.0065 - r_square: 0.9132 - val_loss: 0.0070 - val_acc: 0.7717 - 

Epoch 33/100
35456/35456 [==============================] - 3s 94us/step - loss: 0.0017 - acc: 0.8419 - rmse: 0.0348 - mse: 0.0017 - r_square: 0.9762 - val_loss: 0.0030 - val_acc: 0.7880 - val_rmse: 0.0424 - val_mse: 0.0030 - val_r_square: 0.9706
Epoch 34/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0016 - acc: 0.8523 - rmse: 0.0335 - mse: 0.0016 - r_square: 0.9777 - val_loss: 0.0031 - val_acc: 0.7841 - val_rmse: 0.0419 - val_mse: 0.0031 - val_r_square: 0.9711
Epoch 35/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0016 - acc: 0.8524 - rmse: 0.0338 - mse: 0.0016 - r_square: 0.9771 - val_loss: 0.0031 - val_acc: 0.7837 - val_rmse: 0.0428 - val_mse: 0.0031 - val_r_square: 0.9700
Epoch 36/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0016 - acc: 0.8492 - rmse: 0.0334 - mse: 0.0016 - r_square: 0.9776 - val_loss: 0.0031 - val_acc: 0.7835 - val_rmse: 0.0426 - val_mse: 0.0031 - val_r_square: 0.9703
Epoch 37/100

35456/35456 [==============================] - 3s 95us/step - loss: 0.0012 - acc: 0.8643 - rmse: 0.0282 - mse: 0.0012 - r_square: 0.9830 - val_loss: 0.0042 - val_acc: 0.8943 - val_rmse: 0.0452 - val_mse: 0.0042 - val_r_square: 0.9637
Epoch 100/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0013 - acc: 0.8641 - rmse: 0.0290 - mse: 0.0013 - r_square: 0.9823 - val_loss: 0.0044 - val_acc: 0.8682 - val_rmse: 0.0486 - val_mse: 0.0044 - val_r_square: 0.9615
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 14s 392us/step - loss: 0.0372 - acc: 0.5910 - rmse: 0.1492 - mse: 0.0372 - r_square: 0.5636 - val_loss: 0.0064 - val_acc: 0.7443 - val_rmse: 0.0674 - val_mse: 0.0064 - val_r_square: 0.9375
Epoch 2/100
35456/35456 [==============================] - 5

35456/35456 [==============================] - 5s 152us/step - loss: 0.0012 - acc: 0.8632 - rmse: 0.0275 - mse: 0.0012 - r_square: 0.9832 - val_loss: 0.0025 - val_acc: 0.7917 - val_rmse: 0.0345 - val_mse: 0.0025 - val_r_square: 0.9769
Epoch 65/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0011 - acc: 0.8634 - rmse: 0.0264 - mse: 0.0011 - r_square: 0.9841 - val_loss: 0.0029 - val_acc: 0.7870 - val_rmse: 0.0384 - val_mse: 0.0029 - val_r_square: 0.9728
Epoch 66/100
35456/35456 [==============================] - 5s 145us/step - loss: 0.0012 - acc: 0.8634 - rmse: 0.0277 - mse: 0.0012 - r_square: 0.9829 - val_loss: 0.0028 - val_acc: 0.7896 - val_rmse: 0.0373 - val_mse: 0.0028 - val_r_square: 0.9740
Epoch 67/100
35456/35456 [==============================] - 5s 140us/step - loss: 0.0012 - acc: 0.8628 - rmse: 0.0276 - mse: 0.0012 - r_square: 0.9833 - val_loss: 0.0032 - val_acc: 0.7882 - val_rmse: 0.0399 - val_mse: 0.0032 - val_r_square: 0.9714
Epoch 68/100
35456/35

35456/35456 [==============================] - 5s 143us/step - loss: 0.0033 - acc: 0.8122 - mse: 0.0033 - rmse: 0.0458 - r_square: 0.9527 - val_loss: 0.0119 - val_acc: 0.8539 - val_mse: 0.0119 - val_rmse: 0.0958 - val_r_square: 0.8802
Epoch 30/100
35456/35456 [==============================] - 5s 142us/step - loss: 0.0035 - acc: 0.8204 - mse: 0.0035 - rmse: 0.0476 - r_square: 0.9491 - val_loss: 0.0123 - val_acc: 0.8539 - val_mse: 0.0123 - val_rmse: 0.0981 - val_r_square: 0.8724
Epoch 31/100
35456/35456 [==============================] - 5s 144us/step - loss: 0.0030 - acc: 0.8488 - mse: 0.0030 - rmse: 0.0436 - r_square: 0.9575 - val_loss: 0.0130 - val_acc: 0.8539 - val_mse: 0.0130 - val_rmse: 0.1016 - val_r_square: 0.8641
Epoch 32/100
35456/35456 [==============================] - 5s 145us/step - loss: 0.0039 - acc: 0.8148 - mse: 0.0039 - rmse: 0.0492 - r_square: 0.9456 - val_loss: 0.0112 - val_acc: 0.8539 - val_mse: 0.0112 - val_rmse: 0.0910 - val_r_square: 0.8926
Epoch 33/100
35456/35

35456/35456 [==============================] - 6s 173us/step - loss: 0.0032 - acc: 0.8691 - mse: 0.0032 - rmse: 0.0448 - r_square: 0.9560 - val_loss: 0.0133 - val_acc: 0.8547 - val_mse: 0.0133 - val_rmse: 0.1032 - val_r_square: 0.8608
Epoch 96/100
35456/35456 [==============================] - 6s 176us/step - loss: 0.0034 - acc: 0.8503 - mse: 0.0034 - rmse: 0.0470 - r_square: 0.9519 - val_loss: 0.0133 - val_acc: 0.8540 - val_mse: 0.0133 - val_rmse: 0.1031 - val_r_square: 0.8603
Epoch 97/100
35456/35456 [==============================] - 6s 183us/step - loss: 0.0033 - acc: 0.8730 - mse: 0.0033 - rmse: 0.0456 - r_square: 0.9539 - val_loss: 0.0134 - val_acc: 0.8540 - val_mse: 0.0134 - val_rmse: 0.1031 - val_r_square: 0.8611
Epoch 98/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0033 - acc: 0.8673 - mse: 0.0033 - rmse: 0.0456 - r_square: 0.9549 - val_loss: 0.0132 - val_acc: 0.8540 - val_mse: 0.0132 - val_rmse: 0.1025 - val_r_square: 0.8618
Epoch 99/100
35456/35

# apparentTemperatureLow

In [35]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'apparentTemperatureLow','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 11s 315us/step - loss: 0.0570 - acc: 0.5507 - rmse: 0.2036 - mse: 0.0570 - r_square: 0.2975 - val_loss: 0.0146 - val_acc: 0.8853 - val_rmse: 0.1029 - val_mse: 0.0146 - val_r_square: 0.8462
Epoch 2/100
35456/35456 [==============================] - 3s 95us/step - loss: 0.0100 - acc: 0.6900 - rmse: 0.0895 - mse: 0.0100 - r_square: 0.8678 - val_loss: 0.0077 - val_acc: 0.8853 - val_rmse: 0.0756 - val_mse: 0.0077 - val_r_square: 0.9215
Epoch 3/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0070 - acc: 0.6760 - rmse: 0.0723 - mse: 0.0070 - r_square: 0.9052 - val_loss: 0.0077 - val_acc: 0.8853 - v

Epoch 33/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0017 - acc: 0.8433 - rmse: 0.0352 - mse: 0.0017 - r_square: 0.9762 - val_loss: 0.0033 - val_acc: 0.8208 - val_rmse: 0.0449 - val_mse: 0.0033 - val_r_square: 0.9686
Epoch 34/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0018 - acc: 0.8468 - rmse: 0.0363 - mse: 0.0018 - r_square: 0.9741 - val_loss: 0.0028 - val_acc: 0.8262 - val_rmse: 0.0409 - val_mse: 0.0028 - val_r_square: 0.9725
Epoch 35/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0017 - acc: 0.8437 - rmse: 0.0349 - mse: 0.0017 - r_square: 0.9764 - val_loss: 0.0033 - val_acc: 0.8225 - val_rmse: 0.0445 - val_mse: 0.0033 - val_r_square: 0.9691
Epoch 36/100
35456/35456 [==============================] - 3s 94us/step - loss: 0.0018 - acc: 0.8472 - rmse: 0.0358 - mse: 0.0018 - r_square: 0.9746 - val_loss: 0.0030 - val_acc: 0.8238 - val_rmse: 0.0428 - val_mse: 0.0030 - val_r_square: 0.9709
Epoch 37/100

35456/35456 [==============================] - 3s 93us/step - loss: 0.0014 - acc: 0.8567 - rmse: 0.0302 - mse: 0.0014 - r_square: 0.9810 - val_loss: 0.0042 - val_acc: 0.8916 - val_rmse: 0.0487 - val_mse: 0.0042 - val_r_square: 0.9601
Epoch 100/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0013 - acc: 0.8601 - rmse: 0.0296 - mse: 0.0013 - r_square: 0.9821 - val_loss: 0.0043 - val_acc: 0.8990 - val_rmse: 0.0479 - val_mse: 0.0043 - val_r_square: 0.9607
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 15s 420us/step - loss: 0.0426 - acc: 0.5955 - rmse: 0.1663 - mse: 0.0426 - r_square: 0.4927 - val_loss: 0.0069 - val_acc: 0.8853 - val_rmse: 0.0706 - val_mse: 0.0069 - val_r_square: 0.9335
Epoch 2/100
35456/35456 [==============================] - 5

35456/35456 [==============================] - 5s 139us/step - loss: 0.0013 - acc: 0.8610 - rmse: 0.0293 - mse: 0.0013 - r_square: 0.9821 - val_loss: 0.0034 - val_acc: 0.7883 - val_rmse: 0.0438 - val_mse: 0.0034 - val_r_square: 0.9691
Epoch 65/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0013 - acc: 0.8599 - rmse: 0.0297 - mse: 0.0013 - r_square: 0.9811 - val_loss: 0.0030 - val_acc: 0.7913 - val_rmse: 0.0400 - val_mse: 0.0030 - val_r_square: 0.9723
Epoch 66/100
35456/35456 [==============================] - 5s 141us/step - loss: 0.0013 - acc: 0.8611 - rmse: 0.0289 - mse: 0.0013 - r_square: 0.9825 - val_loss: 0.0033 - val_acc: 0.7941 - val_rmse: 0.0421 - val_mse: 0.0033 - val_r_square: 0.9704
Epoch 67/100
35456/35456 [==============================] - 5s 138us/step - loss: 0.0013 - acc: 0.8500 - rmse: 0.0299 - mse: 0.0013 - r_square: 0.9810 - val_loss: 0.0030 - val_acc: 0.7899 - val_rmse: 0.0397 - val_mse: 0.0030 - val_r_square: 0.9721
Epoch 68/100
35456/35

35456/35456 [==============================] - 5s 146us/step - loss: 0.0024 - acc: 0.8800 - mse: 0.0024 - rmse: 0.0384 - r_square: 0.9671 - val_loss: 0.0103 - val_acc: 0.8844 - val_mse: 0.0103 - val_rmse: 0.0888 - val_r_square: 0.8948
Epoch 30/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0026 - acc: 0.8625 - mse: 0.0026 - rmse: 0.0397 - r_square: 0.9639 - val_loss: 0.0159 - val_acc: 0.8854 - val_mse: 0.0159 - val_rmse: 0.1155 - val_r_square: 0.8185
Epoch 31/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0027 - acc: 0.8562 - mse: 0.0027 - rmse: 0.0412 - r_square: 0.9629 - val_loss: 0.0111 - val_acc: 0.8854 - val_mse: 0.0111 - val_rmse: 0.0944 - val_r_square: 0.8821
Epoch 32/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0025 - acc: 0.8655 - mse: 0.0025 - rmse: 0.0386 - r_square: 0.9658 - val_loss: 0.0107 - val_acc: 0.8854 - val_mse: 0.0107 - val_rmse: 0.0926 - val_r_square: 0.8863
Epoch 33/100
35456/35

35456/35456 [==============================] - 5s 146us/step - loss: 0.0028 - acc: 0.8583 - mse: 0.0028 - rmse: 0.0406 - r_square: 0.9620 - val_loss: 0.0119 - val_acc: 0.8854 - val_mse: 0.0119 - val_rmse: 0.0979 - val_r_square: 0.8706
Epoch 96/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0027 - acc: 0.8593 - mse: 0.0027 - rmse: 0.0394 - r_square: 0.9635 - val_loss: 0.0123 - val_acc: 0.8854 - val_mse: 0.0123 - val_rmse: 0.1004 - val_r_square: 0.8642
Epoch 97/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0026 - acc: 0.8583 - mse: 0.0026 - rmse: 0.0387 - r_square: 0.9644 - val_loss: 0.0120 - val_acc: 0.8854 - val_mse: 0.0120 - val_rmse: 0.0986 - val_r_square: 0.8691
Epoch 98/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0026 - acc: 0.8601 - mse: 0.0026 - rmse: 0.0385 - r_square: 0.9648 - val_loss: 0.0120 - val_acc: 0.8854 - val_mse: 0.0120 - val_rmse: 0.0988 - val_r_square: 0.8688
Epoch 99/100
35456/35

# dewPoint

In [37]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'dewPoint','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 15s 431us/step - loss: 0.0601 - acc: 0.5819 - rmse: 0.2099 - mse: 0.0601 - r_square: 0.2793 - val_loss: 0.0161 - val_acc: 0.7485 - val_rmse: 0.1088 - val_mse: 0.0161 - val_r_square: 0.8333
Epoch 2/100
35456/35456 [==============================] - 4s 117us/step - loss: 0.0129 - acc: 0.6214 - rmse: 0.1016 - mse: 0.0129 - r_square: 0.8401 - val_loss: 0.0094 - val_acc: 0.7825 - val_rmse: 0.0835 - val_mse: 0.0094 - val_r_square: 0.9056
Epoch 3/100
35456/35456 [==============================] - 4s 115us/step - loss: 0.0085 - acc: 0.6947 - rmse: 0.0806 - mse: 0.0085 - r_square: 0.8910 - val_loss: 0.0067 - val_acc: 0.8054 -

Epoch 33/100
35456/35456 [==============================] - 4s 100us/step - loss: 0.0021 - acc: 0.8297 - rmse: 0.0378 - mse: 0.0021 - r_square: 0.9719 - val_loss: 0.0055 - val_acc: 0.8098 - val_rmse: 0.0609 - val_mse: 0.0055 - val_r_square: 0.9452
Epoch 34/100
35456/35456 [==============================] - 4s 107us/step - loss: 0.0022 - acc: 0.8340 - rmse: 0.0389 - mse: 0.0022 - r_square: 0.9703 - val_loss: 0.0051 - val_acc: 0.8412 - val_rmse: 0.0610 - val_mse: 0.0051 - val_r_square: 0.9456
Epoch 35/100
35456/35456 [==============================] - 4s 105us/step - loss: 0.0033 - acc: 0.8001 - rmse: 0.0479 - mse: 0.0033 - r_square: 0.9572 - val_loss: 0.0086 - val_acc: 0.7169 - val_rmse: 0.0828 - val_mse: 0.0086 - val_r_square: 0.9009
Epoch 36/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0044 - acc: 0.8241 - rmse: 0.0561 - mse: 0.0044 - r_square: 0.9424 - val_loss: 0.0086 - val_acc: 0.8269 - val_rmse: 0.0820 - val_mse: 0.0086 - val_r_square: 0.8958
Epoch 37/

35456/35456 [==============================] - 3s 95us/step - loss: 0.0015 - acc: 0.8809 - rmse: 0.0313 - mse: 0.0015 - r_square: 0.9795 - val_loss: 0.0060 - val_acc: 0.8289 - val_rmse: 0.0528 - val_mse: 0.0060 - val_r_square: 0.9479
Epoch 100/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0015 - acc: 0.8326 - rmse: 0.0308 - mse: 0.0015 - r_square: 0.9794 - val_loss: 0.0058 - val_acc: 0.8184 - val_rmse: 0.0514 - val_mse: 0.0058 - val_r_square: 0.9499
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 18s 518us/step - loss: 0.0371 - acc: 0.5775 - rmse: 0.1515 - mse: 0.0371 - r_square: 0.5802 - val_loss: 0.0065 - val_acc: 0.8002 - val_rmse: 0.0696 - val_mse: 0.0065 - val_r_square: 0.9314
Epoch 2/100
35456/35456 [==============================] - 5

35456/35456 [==============================] - 5s 150us/step - loss: 0.0017 - acc: 0.8458 - rmse: 0.0336 - mse: 0.0017 - r_square: 0.9770 - val_loss: 0.0052 - val_acc: 0.7216 - val_rmse: 0.0491 - val_mse: 0.0052 - val_r_square: 0.9551
Epoch 65/100
35456/35456 [==============================] - 5s 151us/step - loss: 0.0018 - acc: 0.8574 - rmse: 0.0347 - mse: 0.0018 - r_square: 0.9747 - val_loss: 0.0046 - val_acc: 0.7388 - val_rmse: 0.0448 - val_mse: 0.0046 - val_r_square: 0.9602
Epoch 66/100
35456/35456 [==============================] - 5s 150us/step - loss: 0.0017 - acc: 0.8493 - rmse: 0.0336 - mse: 0.0017 - r_square: 0.9771 - val_loss: 0.0052 - val_acc: 0.7213 - val_rmse: 0.0486 - val_mse: 0.0052 - val_r_square: 0.9559
Epoch 67/100
35456/35456 [==============================] - 5s 150us/step - loss: 0.0017 - acc: 0.8591 - rmse: 0.0339 - mse: 0.0017 - r_square: 0.9758 - val_loss: 0.0046 - val_acc: 0.7355 - val_rmse: 0.0439 - val_mse: 0.0046 - val_r_square: 0.9611
Epoch 68/100
35456/35

35456/35456 [==============================] - 6s 156us/step - loss: 0.0033 - acc: 0.8393 - mse: 0.0033 - rmse: 0.0442 - r_square: 0.9572 - val_loss: 0.0139 - val_acc: 0.8058 - val_mse: 0.0139 - val_rmse: 0.1049 - val_r_square: 0.8564
Epoch 30/100
35456/35456 [==============================] - 5s 154us/step - loss: 0.0031 - acc: 0.8631 - mse: 0.0031 - rmse: 0.0426 - r_square: 0.9618 - val_loss: 0.0171 - val_acc: 0.8058 - val_mse: 0.0171 - val_rmse: 0.1194 - val_r_square: 0.8090
Epoch 31/100
35456/35456 [==============================] - 5s 155us/step - loss: 0.0031 - acc: 0.8528 - mse: 0.0031 - rmse: 0.0434 - r_square: 0.9606 - val_loss: 0.0182 - val_acc: 0.8058 - val_mse: 0.0182 - val_rmse: 0.1245 - val_r_square: 0.7946
Epoch 32/100
35456/35456 [==============================] - 6s 156us/step - loss: 0.0028 - acc: 0.8603 - mse: 0.0028 - rmse: 0.0421 - r_square: 0.9636 - val_loss: 0.0123 - val_acc: 0.8058 - val_mse: 0.0123 - val_rmse: 0.0973 - val_r_square: 0.8763
Epoch 33/100
35456/35

35456/35456 [==============================] - 5s 153us/step - loss: 0.0018 - acc: 0.8733 - mse: 0.0018 - rmse: 0.0340 - r_square: 0.9745 - val_loss: 0.0094 - val_acc: 0.8058 - val_mse: 0.0094 - val_rmse: 0.0792 - val_r_square: 0.9136
Epoch 96/100
35456/35456 [==============================] - 6s 157us/step - loss: 0.0019 - acc: 0.8690 - mse: 0.0019 - rmse: 0.0349 - r_square: 0.9733 - val_loss: 0.0086 - val_acc: 0.8058 - val_mse: 0.0086 - val_rmse: 0.0755 - val_r_square: 0.9186
Epoch 97/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0019 - acc: 0.8653 - mse: 0.0019 - rmse: 0.0354 - r_square: 0.9727 - val_loss: 0.0091 - val_acc: 0.8058 - val_mse: 0.0091 - val_rmse: 0.0785 - val_r_square: 0.9159
Epoch 98/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0017 - acc: 0.8719 - mse: 0.0017 - rmse: 0.0328 - r_square: 0.9758 - val_loss: 0.0099 - val_acc: 0.8050 - val_mse: 0.0099 - val_rmse: 0.0797 - val_r_square: 0.9101
Epoch 99/100
35456/35

# pressure

In [40]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'pressure','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 17s 469us/step - loss: 0.0555 - acc: 0.5851 - rmse: 0.1981 - mse: 0.0555 - r_square: 0.3126 - val_loss: 0.0151 - val_acc: 0.8015 - val_rmse: 0.1053 - val_mse: 0.0151 - val_r_square: 0.8437
Epoch 2/100
35456/35456 [==============================] - 5s 129us/step - loss: 0.0101 - acc: 0.7153 - rmse: 0.0886 - mse: 0.0101 - r_square: 0.8711 - val_loss: 0.0101 - val_acc: 0.8241 - val_rmse: 0.0857 - val_mse: 0.0101 - val_r_square: 0.8995
Epoch 3/100
35456/35456 [==============================] - 4s 115us/step - loss: 0.0067 - acc: 0.7147 - rmse: 0.0711 - mse: 0.0067 - r_square: 0.9096 - val_loss: 0.0084 - val_acc: 0.8515 -

Epoch 33/100
35456/35456 [==============================] - 4s 107us/step - loss: 0.0027 - acc: 0.7770 - rmse: 0.0443 - mse: 0.0027 - r_square: 0.9648 - val_loss: 0.0048 - val_acc: 0.8515 - val_rmse: 0.0585 - val_mse: 0.0048 - val_r_square: 0.9476
Epoch 34/100
35456/35456 [==============================] - 4s 101us/step - loss: 0.0027 - acc: 0.7784 - rmse: 0.0440 - mse: 0.0027 - r_square: 0.9652 - val_loss: 0.0048 - val_acc: 0.8515 - val_rmse: 0.0578 - val_mse: 0.0048 - val_r_square: 0.9482
Epoch 35/100
35456/35456 [==============================] - 4s 102us/step - loss: 0.0026 - acc: 0.7798 - rmse: 0.0432 - mse: 0.0026 - r_square: 0.9661 - val_loss: 0.0047 - val_acc: 0.8515 - val_rmse: 0.0568 - val_mse: 0.0047 - val_r_square: 0.9491
Epoch 36/100
35456/35456 [==============================] - 4s 101us/step - loss: 0.0025 - acc: 0.7837 - rmse: 0.0422 - mse: 0.0025 - r_square: 0.9672 - val_loss: 0.0046 - val_acc: 0.8145 - val_rmse: 0.0566 - val_mse: 0.0046 - val_r_square: 0.9494
Epoch 37

35456/35456 [==============================] - 3s 98us/step - loss: 0.0019 - acc: 0.8015 - rmse: 0.0367 - mse: 0.0019 - r_square: 0.9734 - val_loss: 0.0056 - val_acc: 0.7433 - val_rmse: 0.0637 - val_mse: 0.0056 - val_r_square: 0.9406
Epoch 100/100
35456/35456 [==============================] - 3s 98us/step - loss: 0.0022 - acc: 0.8202 - rmse: 0.0390 - mse: 0.0022 - r_square: 0.9714 - val_loss: 0.0066 - val_acc: 0.7812 - val_rmse: 0.0695 - val_mse: 0.0066 - val_r_square: 0.9280
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 19s 534us/step - loss: 0.0387 - acc: 0.6113 - rmse: 0.1479 - mse: 0.0387 - r_square: 0.5383 - val_loss: 0.0083 - val_acc: 0.8515 - val_rmse: 0.0786 - val_mse: 0.0083 - val_r_square: 0.9132
Epoch 2/100
35456/35456 [==============================] - 5

35456/35456 [==============================] - 5s 144us/step - loss: 0.0019 - acc: 0.8046 - rmse: 0.0368 - mse: 0.0019 - r_square: 0.9733 - val_loss: 0.0038 - val_acc: 0.7706 - val_rmse: 0.0506 - val_mse: 0.0038 - val_r_square: 0.9597
Epoch 65/100
35456/35456 [==============================] - 5s 143us/step - loss: 0.0021 - acc: 0.7979 - rmse: 0.0383 - mse: 0.0021 - r_square: 0.9722 - val_loss: 0.0043 - val_acc: 0.7464 - val_rmse: 0.0563 - val_mse: 0.0043 - val_r_square: 0.9524
Epoch 66/100
35456/35456 [==============================] - 5s 143us/step - loss: 0.0021 - acc: 0.8095 - rmse: 0.0393 - mse: 0.0021 - r_square: 0.9708 - val_loss: 0.0041 - val_acc: 0.7542 - val_rmse: 0.0535 - val_mse: 0.0041 - val_r_square: 0.9569
Epoch 67/100
35456/35456 [==============================] - 5s 142us/step - loss: 0.0020 - acc: 0.8161 - rmse: 0.0383 - mse: 0.0020 - r_square: 0.9725 - val_loss: 0.0043 - val_acc: 0.7488 - val_rmse: 0.0551 - val_mse: 0.0043 - val_r_square: 0.9539
Epoch 68/100
35456/35

35456/35456 [==============================] - 5s 146us/step - loss: 0.0025 - acc: 0.8323 - mse: 0.0025 - rmse: 0.0405 - r_square: 0.9648 - val_loss: 0.0084 - val_acc: 0.8516 - val_mse: 0.0084 - val_rmse: 0.0788 - val_r_square: 0.9096
Epoch 30/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0024 - acc: 0.8244 - mse: 0.0024 - rmse: 0.0398 - r_square: 0.9671 - val_loss: 0.0092 - val_acc: 0.8436 - val_mse: 0.0092 - val_rmse: 0.0812 - val_r_square: 0.9020
Epoch 31/100
35456/35456 [==============================] - 5s 146us/step - loss: 0.0024 - acc: 0.8226 - mse: 0.0024 - rmse: 0.0400 - r_square: 0.9669 - val_loss: 0.0092 - val_acc: 0.8515 - val_mse: 0.0092 - val_rmse: 0.0828 - val_r_square: 0.9019
Epoch 32/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0024 - acc: 0.8230 - mse: 0.0024 - rmse: 0.0395 - r_square: 0.9681 - val_loss: 0.0095 - val_acc: 0.8491 - val_mse: 0.0095 - val_rmse: 0.0819 - val_r_square: 0.8993
Epoch 33/100
35456/35

35456/35456 [==============================] - 5s 147us/step - loss: 0.0020 - acc: 0.8319 - mse: 0.0020 - rmse: 0.0364 - r_square: 0.9723 - val_loss: 0.0081 - val_acc: 0.8516 - val_mse: 0.0081 - val_rmse: 0.0736 - val_r_square: 0.9152
Epoch 96/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0020 - acc: 0.8287 - mse: 0.0020 - rmse: 0.0364 - r_square: 0.9720 - val_loss: 0.0081 - val_acc: 0.8516 - val_mse: 0.0081 - val_rmse: 0.0722 - val_r_square: 0.9155
Epoch 97/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0019 - acc: 0.8321 - mse: 0.0019 - rmse: 0.0359 - r_square: 0.9725 - val_loss: 0.0083 - val_acc: 0.8516 - val_mse: 0.0083 - val_rmse: 0.0738 - val_r_square: 0.9155
Epoch 98/100
35456/35456 [==============================] - 5s 147us/step - loss: 0.0021 - acc: 0.8273 - mse: 0.0021 - rmse: 0.0381 - r_square: 0.9691 - val_loss: 0.0084 - val_acc: 0.8516 - val_mse: 0.0084 - val_rmse: 0.0730 - val_r_square: 0.9137
Epoch 99/100
35456/35

# windSpeed

In [42]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'windSpeed','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 19s 545us/step - loss: 0.0438 - acc: 0.7934 - rmse: 0.1743 - mse: 0.0438 - r_square: 0.4359 - val_loss: 0.0125 - val_acc: 0.8890 - val_rmse: 0.0950 - val_mse: 0.0125 - val_r_square: 0.8509
Epoch 2/100
35456/35456 [==============================] - 4s 109us/step - loss: 0.0085 - acc: 0.8673 - rmse: 0.0812 - mse: 0.0085 - r_square: 0.8741 - val_loss: 0.0081 - val_acc: 0.9710 - val_rmse: 0.0760 - val_mse: 0.0081 - val_r_square: 0.9057
Epoch 3/100
35456/35456 [==============================] - 4s 109us/step - loss: 0.0067 - acc: 0.8500 - rmse: 0.0714 - mse: 0.0067 - r_square: 0.8939 - val_loss: 0.0079 - val_acc: 0.9710 -

Epoch 33/100
35456/35456 [==============================] - 4s 99us/step - loss: 0.0027 - acc: 0.9321 - rmse: 0.0418 - mse: 0.0027 - r_square: 0.9591 - val_loss: 0.0031 - val_acc: 0.8890 - val_rmse: 0.0456 - val_mse: 0.0031 - val_r_square: 0.9605
Epoch 34/100
35456/35456 [==============================] - 4s 101us/step - loss: 0.0026 - acc: 0.9347 - rmse: 0.0416 - mse: 0.0026 - r_square: 0.9596 - val_loss: 0.0031 - val_acc: 0.8890 - val_rmse: 0.0459 - val_mse: 0.0031 - val_r_square: 0.9601
Epoch 35/100
35456/35456 [==============================] - 4s 99us/step - loss: 0.0026 - acc: 0.9361 - rmse: 0.0414 - mse: 0.0026 - r_square: 0.9600 - val_loss: 0.0031 - val_acc: 0.8891 - val_rmse: 0.0462 - val_mse: 0.0031 - val_r_square: 0.9599
Epoch 36/100
35456/35456 [==============================] - 4s 99us/step - loss: 0.0026 - acc: 0.9376 - rmse: 0.0413 - mse: 0.0026 - r_square: 0.9602 - val_loss: 0.0031 - val_acc: 0.8892 - val_rmse: 0.0463 - val_mse: 0.0031 - val_r_square: 0.9597
Epoch 37/10

35456/35456 [==============================] - 4s 99us/step - loss: 0.0020 - acc: 0.9370 - rmse: 0.0344 - mse: 0.0020 - r_square: 0.9683 - val_loss: 0.0035 - val_acc: 0.9342 - val_rmse: 0.0449 - val_mse: 0.0035 - val_r_square: 0.9590
Epoch 100/100
35456/35456 [==============================] - 4s 100us/step - loss: 0.0020 - acc: 0.9387 - rmse: 0.0351 - mse: 0.0020 - r_square: 0.9674 - val_loss: 0.0038 - val_acc: 0.9550 - val_rmse: 0.0489 - val_mse: 0.0038 - val_r_square: 0.9539
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 23s 640us/step - loss: 0.0336 - acc: 0.7690 - rmse: 0.1412 - mse: 0.0336 - r_square: 0.5780 - val_loss: 0.0060 - val_acc: 0.9984 - val_rmse: 0.0646 - val_mse: 0.0060 - val_r_square: 0.9285
Epoch 2/100
35456/35456 [==============================] - 

35456/35456 [==============================] - 5s 146us/step - loss: 0.0020 - acc: 0.9539 - rmse: 0.0357 - mse: 0.0020 - r_square: 0.9669 - val_loss: 0.0036 - val_acc: 0.8806 - val_rmse: 0.0504 - val_mse: 0.0036 - val_r_square: 0.9550
Epoch 65/100
35456/35456 [==============================] - 5s 144us/step - loss: 0.0021 - acc: 0.9539 - rmse: 0.0370 - mse: 0.0021 - r_square: 0.9657 - val_loss: 0.0034 - val_acc: 0.8806 - val_rmse: 0.0467 - val_mse: 0.0034 - val_r_square: 0.9601
Epoch 66/100
35456/35456 [==============================] - 5s 145us/step - loss: 0.0022 - acc: 0.9444 - rmse: 0.0371 - mse: 0.0022 - r_square: 0.9653 - val_loss: 0.0031 - val_acc: 0.8806 - val_rmse: 0.0427 - val_mse: 0.0031 - val_r_square: 0.9641
Epoch 67/100
35456/35456 [==============================] - 5s 145us/step - loss: 0.0021 - acc: 0.9434 - rmse: 0.0358 - mse: 0.0021 - r_square: 0.9666 - val_loss: 0.0028 - val_acc: 0.8806 - val_rmse: 0.0400 - val_mse: 0.0028 - val_r_square: 0.9662
Epoch 68/100
35456/35

35456/35456 [==============================] - 5s 153us/step - loss: 0.0041 - acc: 0.9604 - mse: 0.0041 - rmse: 0.0458 - r_square: 0.8841 - val_loss: 0.0091 - val_acc: 0.9984 - val_mse: 0.0091 - val_rmse: 0.0830 - val_r_square: 0.8900
Epoch 30/100
35456/35456 [==============================] - 5s 155us/step - loss: 0.0050 - acc: 0.9664 - mse: 0.0050 - rmse: 0.0491 - r_square: 0.8360 - val_loss: 0.0111 - val_acc: 0.9984 - val_mse: 0.0111 - val_rmse: 0.0937 - val_r_square: 0.8684
Epoch 31/100
35456/35456 [==============================] - 5s 154us/step - loss: 0.0035 - acc: 0.9656 - mse: 0.0035 - rmse: 0.0461 - r_square: 0.9297 - val_loss: 0.0094 - val_acc: 0.9984 - val_mse: 0.0094 - val_rmse: 0.0855 - val_r_square: 0.8805
Epoch 32/100
35456/35456 [==============================] - 5s 155us/step - loss: 0.0028 - acc: 0.9790 - mse: 0.0028 - rmse: 0.0419 - r_square: 0.9551 - val_loss: 0.0082 - val_acc: 0.9984 - val_mse: 0.0082 - val_rmse: 0.0780 - val_r_square: 0.8982
Epoch 33/100
35456/35

35456/35456 [==============================] - 5s 153us/step - loss: 0.0044 - acc: 0.9681 - mse: 0.0044 - rmse: 0.0513 - r_square: 0.9125 - val_loss: 0.0097 - val_acc: 0.9985 - val_mse: 0.0097 - val_rmse: 0.0838 - val_r_square: 0.8875
Epoch 96/100
35456/35456 [==============================] - 5s 153us/step - loss: 0.0043 - acc: 0.9726 - mse: 0.0043 - rmse: 0.0511 - r_square: 0.9158 - val_loss: 0.0099 - val_acc: 0.9985 - val_mse: 0.0099 - val_rmse: 0.0844 - val_r_square: 0.8839
Epoch 97/100
35456/35456 [==============================] - 5s 154us/step - loss: 0.0044 - acc: 0.9725 - mse: 0.0044 - rmse: 0.0513 - r_square: 0.9160 - val_loss: 0.0135 - val_acc: 0.9985 - val_mse: 0.0135 - val_rmse: 0.1061 - val_r_square: 0.8215
Epoch 98/100
35456/35456 [==============================] - 5s 154us/step - loss: 0.0042 - acc: 0.9729 - mse: 0.0042 - rmse: 0.0503 - r_square: 0.9127 - val_loss: 0.0135 - val_acc: 0.9985 - val_mse: 0.0135 - val_rmse: 0.1064 - val_r_square: 0.8224
Epoch 99/100
35456/35

# cloudCover

In [10]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'cloudCover','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 6s 173us/step - loss: 0.0740 - acc: 0.4295 - rmse: 0.2409 - mse: 0.0740 - r_square: 0.2289 - val_loss: 0.0304 - val_acc: 0.6024 - val_rmse: 0.1589 - val_mse: 0.0304 - val_r_square: 0.6869
Epoch 2/100
35456/35456 [==============================] - 3s 96us/step - loss: 0.0261 - acc: 0.5083 - rmse: 0.1480 - mse: 0.0261 - r_square: 0.7079 - val_loss: 0.0236 - val_acc: 0.6450 - val_rmse: 0.1396 - val_mse: 0.0236 - val_r_square: 0.7532
Epoch 3/100
35456/35456 [==============================] - 3s 94us/ste

35456/35456 [==============================] - 3s 90us/step - loss: 0.0182 - acc: 0.5655 - rmse: 0.1191 - mse: 0.0182 - r_square: 0.7896 - val_loss: 0.0207 - val_acc: 0.6635 - val_rmse: 0.1254 - val_mse: 0.0207 - val_r_square: 0.7834
Epoch 64/100
35456/35456 [==============================] - 3s 89us/step - loss: 0.0182 - acc: 0.5654 - rmse: 0.1193 - mse: 0.0182 - r_square: 0.7893 - val_loss: 0.0208 - val_acc: 0.6661 - val_rmse: 0.1258 - val_mse: 0.0208 - val_r_square: 0.7821
Epoch 65/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0183 - acc: 0.5648 - rmse: 0.1194 - mse: 0.0183 - r_square: 0.7893 - val_loss: 0.0210 - val_acc: 0.6671 - val_rmse: 0.1267 - val_mse: 0.0210 - val_r_square: 0.7798
Epoch 66/100
35456/35456 [==============================] - 3s 89us/step - loss: 0.0183 - acc: 0.5651 - rmse: 0.1196 - mse: 0.0183 - r_square: 0.7891 - val_loss: 0.0212 - val_acc: 0.6674 - val_rmse: 0.1277 - val_mse: 0.0212 - val_r_square: 0.7773
Epoch 67/100
35456/35456 

35456/35456 [==============================] - 5s 135us/step - loss: 0.0185 - acc: 0.5485 - rmse: 0.1202 - mse: 0.0185 - r_square: 0.7866 - val_loss: 0.0228 - val_acc: 0.5905 - val_rmse: 0.1376 - val_mse: 0.0228 - val_r_square: 0.7554
Epoch 29/100
35456/35456 [==============================] - 5s 135us/step - loss: 0.0185 - acc: 0.5527 - rmse: 0.1201 - mse: 0.0185 - r_square: 0.7871 - val_loss: 0.0220 - val_acc: 0.5926 - val_rmse: 0.1353 - val_mse: 0.0220 - val_r_square: 0.7633
Epoch 30/100
35456/35456 [==============================] - 5s 135us/step - loss: 0.0183 - acc: 0.5530 - rmse: 0.1196 - mse: 0.0183 - r_square: 0.7883 - val_loss: 0.0213 - val_acc: 0.5918 - val_rmse: 0.1332 - val_mse: 0.0213 - val_r_square: 0.7702
Epoch 31/100
35456/35456 [==============================] - 5s 134us/step - loss: 0.0182 - acc: 0.5522 - rmse: 0.1189 - mse: 0.0182 - r_square: 0.7897 - val_loss: 0.0206 - val_acc: 0.5897 - val_rmse: 0.1305 - val_mse: 0.0206 - val_r_square: 0.7779
Epoch 32/100
35456/35

35456/35456 [==============================] - 5s 132us/step - loss: 0.0178 - acc: 0.6038 - rmse: 0.1175 - mse: 0.0178 - r_square: 0.7928 - val_loss: 0.0218 - val_acc: 0.6407 - val_rmse: 0.1283 - val_mse: 0.0218 - val_r_square: 0.7754
Epoch 95/100
35456/35456 [==============================] - 5s 132us/step - loss: 0.0178 - acc: 0.5888 - rmse: 0.1172 - mse: 0.0178 - r_square: 0.7936 - val_loss: 0.0228 - val_acc: 0.6474 - val_rmse: 0.1308 - val_mse: 0.0228 - val_r_square: 0.7685
Epoch 96/100
35456/35456 [==============================] - 5s 132us/step - loss: 0.0179 - acc: 0.6032 - rmse: 0.1179 - mse: 0.0179 - r_square: 0.7920 - val_loss: 0.0220 - val_acc: 0.6566 - val_rmse: 0.1297 - val_mse: 0.0220 - val_r_square: 0.7725
Epoch 97/100
35456/35456 [==============================] - 5s 133us/step - loss: 0.0179 - acc: 0.5875 - rmse: 0.1176 - mse: 0.0179 - r_square: 0.7929 - val_loss: 0.0230 - val_acc: 0.6498 - val_rmse: 0.1329 - val_mse: 0.0230 - val_r_square: 0.7639
Epoch 98/100
35456/35

35456/35456 [==============================] - 5s 129us/step - loss: 0.0213 - acc: 0.5597 - mse: 0.0213 - rmse: 0.1287 - r_square: 0.7493 - val_loss: 0.0305 - val_acc: 0.6528 - val_mse: 0.0305 - val_rmse: 0.1617 - val_r_square: 0.6767
Epoch 60/100
35456/35456 [==============================] - 5s 129us/step - loss: 0.0213 - acc: 0.5568 - mse: 0.0213 - rmse: 0.1289 - r_square: 0.7489 - val_loss: 0.0311 - val_acc: 0.6528 - val_mse: 0.0311 - val_rmse: 0.1633 - val_r_square: 0.6747
Epoch 61/100
35456/35456 [==============================] - 5s 131us/step - loss: 0.0213 - acc: 0.5626 - mse: 0.0213 - rmse: 0.1286 - r_square: 0.7497 - val_loss: 0.0307 - val_acc: 0.6528 - val_mse: 0.0307 - val_rmse: 0.1621 - val_r_square: 0.6762
Epoch 62/100
35456/35456 [==============================] - 5s 130us/step - loss: 0.0212 - acc: 0.5614 - mse: 0.0212 - rmse: 0.1285 - r_square: 0.7497 - val_loss: 0.0305 - val_acc: 0.6528 - val_mse: 0.0305 - val_rmse: 0.1617 - val_r_square: 0.6766
Epoch 63/100
35456/35

# uvIndex

In [13]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'uvIndex','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 6s 172us/step - loss: 0.0534 - acc: 0.6966 - rmse: 0.1933 - mse: 0.0534 - r_square: 0.2735 - val_loss: 0.0156 - val_acc: 0.9115 - val_rmse: 0.1090 - val_mse: 0.0156 - val_r_square: 0.7993
Epoch 2/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0096 - acc: 0.8028 - rmse: 0.0866 - mse: 0.0096 - r_square: 0.8569 - val_loss: 0.0099 - val_acc: 0.9042 - val_rmse: 0.0863 - val_mse: 0.0099 - val_r_square: 0.8784
Epoch 3/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0071 - acc: 0.8013 - rmse: 0.0723 - mse: 0.0071 - r_square: 0.8958 - val_loss: 0.0083 - val_acc: 0.8961 - va

Epoch 33/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0020 - acc: 0.9325 - rmse: 0.0368 - mse: 0.0020 - r_square: 0.9692 - val_loss: 0.0035 - val_acc: 0.9664 - val_rmse: 0.0496 - val_mse: 0.0035 - val_r_square: 0.9571
Epoch 34/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0020 - acc: 0.9314 - rmse: 0.0367 - mse: 0.0020 - r_square: 0.9693 - val_loss: 0.0034 - val_acc: 0.9666 - val_rmse: 0.0488 - val_mse: 0.0034 - val_r_square: 0.9583
Epoch 35/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0020 - acc: 0.9287 - rmse: 0.0370 - mse: 0.0020 - r_square: 0.9689 - val_loss: 0.0034 - val_acc: 0.9666 - val_rmse: 0.0491 - val_mse: 0.0034 - val_r_square: 0.9579
Epoch 36/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0021 - acc: 0.9226 - rmse: 0.0382 - mse: 0.0021 - r_square: 0.9671 - val_loss: 0.0036 - val_acc: 0.9666 - val_rmse: 0.0508 - val_mse: 0.0036 - val_r_square: 0.9556
Epoch 37/100

35456/35456 [==============================] - 3s 91us/step - loss: 0.0016 - acc: 0.9438 - rmse: 0.0314 - mse: 0.0016 - r_square: 0.9757 - val_loss: 0.0044 - val_acc: 0.9537 - val_rmse: 0.0495 - val_mse: 0.0044 - val_r_square: 0.9534
Epoch 100/100
35456/35456 [==============================] - 3s 90us/step - loss: 0.0017 - acc: 0.9409 - rmse: 0.0328 - mse: 0.0017 - r_square: 0.9733 - val_loss: 0.0041 - val_acc: 0.8806 - val_rmse: 0.0500 - val_mse: 0.0041 - val_r_square: 0.9541
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 9s 252us/step - loss: 0.0368 - acc: 0.7713 - rmse: 0.1521 - mse: 0.0368 - r_square: 0.5344 - val_loss: 0.0098 - val_acc: 0.9663 - val_rmse: 0.0878 - val_mse: 0.0098 - val_r_square: 0.8742
Epoch 2/100
35456/35456 [==============================] - 5s

35456/35456 [==============================] - 5s 135us/step - loss: 0.0017 - acc: 0.9374 - rmse: 0.0327 - mse: 0.0017 - r_square: 0.9735 - val_loss: 0.0035 - val_acc: 0.8911 - val_rmse: 0.0435 - val_mse: 0.0035 - val_r_square: 0.9615
Epoch 65/100
35456/35456 [==============================] - 5s 132us/step - loss: 0.0015 - acc: 0.9533 - rmse: 0.0303 - mse: 0.0015 - r_square: 0.9762 - val_loss: 0.0038 - val_acc: 0.8968 - val_rmse: 0.0431 - val_mse: 0.0038 - val_r_square: 0.9611
Epoch 66/100
35456/35456 [==============================] - 5s 132us/step - loss: 0.0016 - acc: 0.9321 - rmse: 0.0319 - mse: 0.0016 - r_square: 0.9745 - val_loss: 0.0035 - val_acc: 0.8567 - val_rmse: 0.0419 - val_mse: 0.0035 - val_r_square: 0.9627
Epoch 67/100
35456/35456 [==============================] - 5s 132us/step - loss: 0.0015 - acc: 0.9514 - rmse: 0.0307 - mse: 0.0015 - r_square: 0.9760 - val_loss: 0.0039 - val_acc: 0.8629 - val_rmse: 0.0435 - val_mse: 0.0039 - val_r_square: 0.9602
Epoch 68/100
35456/35

35456/35456 [==============================] - 5s 133us/step - loss: 0.0035 - acc: 0.9632 - mse: 0.0035 - rmse: 0.0479 - r_square: 0.9405 - val_loss: 0.0138 - val_acc: 0.9664 - val_mse: 0.0138 - val_rmse: 0.1048 - val_r_square: 0.8226
Epoch 30/100
35456/35456 [==============================] - 5s 134us/step - loss: 0.0031 - acc: 0.9595 - mse: 0.0031 - rmse: 0.0439 - r_square: 0.9529 - val_loss: 0.0117 - val_acc: 0.9664 - val_mse: 0.0117 - val_rmse: 0.0957 - val_r_square: 0.8528
Epoch 31/100
35456/35456 [==============================] - 5s 131us/step - loss: 0.0031 - acc: 0.9651 - mse: 0.0031 - rmse: 0.0449 - r_square: 0.9474 - val_loss: 0.0129 - val_acc: 0.9664 - val_mse: 0.0129 - val_rmse: 0.1017 - val_r_square: 0.8327
Epoch 32/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0033 - acc: 0.9567 - mse: 0.0033 - rmse: 0.0451 - r_square: 0.9500 - val_loss: 0.0124 - val_acc: 0.9664 - val_mse: 0.0124 - val_rmse: 0.0990 - val_r_square: 0.8436
Epoch 33/100
35456/35

35456/35456 [==============================] - 5s 131us/step - loss: 0.0032 - acc: 0.9570 - mse: 0.0032 - rmse: 0.0454 - r_square: 0.9512 - val_loss: 0.0136 - val_acc: 0.9663 - val_mse: 0.0136 - val_rmse: 0.1046 - val_r_square: 0.8185
Epoch 96/100
35456/35456 [==============================] - 5s 131us/step - loss: 0.0034 - acc: 0.9473 - mse: 0.0034 - rmse: 0.0464 - r_square: 0.9487 - val_loss: 0.0127 - val_acc: 0.9664 - val_mse: 0.0127 - val_rmse: 0.0998 - val_r_square: 0.8321
Epoch 97/100
35456/35456 [==============================] - 5s 131us/step - loss: 0.0035 - acc: 0.9478 - mse: 0.0035 - rmse: 0.0474 - r_square: 0.9399 - val_loss: 0.0141 - val_acc: 0.9664 - val_mse: 0.0141 - val_rmse: 0.1061 - val_r_square: 0.8128
Epoch 98/100
35456/35456 [==============================] - 5s 131us/step - loss: 0.0036 - acc: 0.9510 - mse: 0.0036 - rmse: 0.0484 - r_square: 0.9416 - val_loss: 0.0133 - val_acc: 0.9664 - val_mse: 0.0133 - val_rmse: 0.1024 - val_r_square: 0.8250
Epoch 99/100
35456/35

# ozone

In [15]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'ozone','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 7s 190us/step - loss: 0.0519 - acc: 0.6846 - rmse: 0.1931 - mse: 0.0519 - r_square: 0.3178 - val_loss: 0.0139 - val_acc: 0.9568 - val_rmse: 0.1033 - val_mse: 0.0139 - val_r_square: 0.8245
Epoch 2/100
35456/35456 [==============================] - 4s 102us/step - loss: 0.0093 - acc: 0.8299 - rmse: 0.0860 - mse: 0.0093 - r_square: 0.8615 - val_loss: 0.0097 - val_acc: 0.9842 - val_rmse: 0.0870 - val_mse: 0.0097 - val_r_square: 0.8768
Epoch 3/100
35456/35456 [==============================] - 4s 101us/step - loss: 0.0068 - acc: 0.8273 - rmse: 0.0732 - mse: 0.0068 - r_square: 0.8950 - val_loss: 0.0078 - val_acc: 0.9841 - 

Epoch 33/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0033 - acc: 0.8721 - rmse: 0.0501 - mse: 0.0033 - r_square: 0.9519 - val_loss: 0.0061 - val_acc: 0.8661 - val_rmse: 0.0696 - val_mse: 0.0061 - val_r_square: 0.9229
Epoch 34/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0035 - acc: 0.8745 - rmse: 0.0508 - mse: 0.0035 - r_square: 0.9509 - val_loss: 0.0056 - val_acc: 0.8661 - val_rmse: 0.0648 - val_mse: 0.0056 - val_r_square: 0.9316
Epoch 35/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0033 - acc: 0.8704 - rmse: 0.0492 - mse: 0.0033 - r_square: 0.9535 - val_loss: 0.0050 - val_acc: 0.8661 - val_rmse: 0.0591 - val_mse: 0.0050 - val_r_square: 0.9400
Epoch 36/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0031 - acc: 0.8703 - rmse: 0.0467 - mse: 0.0031 - r_square: 0.9571 - val_loss: 0.0046 - val_acc: 0.8661 - val_rmse: 0.0550 - val_mse: 0.0046 - val_r_square: 0.9454
Epoch 37/100

35456/35456 [==============================] - 3s 91us/step - loss: 0.0018 - acc: 0.8962 - rmse: 0.0334 - mse: 0.0018 - r_square: 0.9712 - val_loss: 0.0048 - val_acc: 0.9756 - val_rmse: 0.0503 - val_mse: 0.0048 - val_r_square: 0.9519
Epoch 100/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0017 - acc: 0.8997 - rmse: 0.0332 - mse: 0.0017 - r_square: 0.9722 - val_loss: 0.0050 - val_acc: 0.9754 - val_rmse: 0.0516 - val_mse: 0.0050 - val_r_square: 0.9501
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 10s 283us/step - loss: 0.0337 - acc: 0.7415 - rmse: 0.1479 - mse: 0.0337 - r_square: 0.5853 - val_loss: 0.0089 - val_acc: 0.9842 - val_rmse: 0.0831 - val_mse: 0.0089 - val_r_square: 0.8875
Epoch 2/100
35456/35456 [==============================] - 5

35456/35456 [==============================] - 5s 136us/step - loss: 0.0021 - acc: 0.8702 - rmse: 0.0378 - mse: 0.0021 - r_square: 0.9684 - val_loss: 0.0052 - val_acc: 0.9757 - val_rmse: 0.0572 - val_mse: 0.0052 - val_r_square: 0.9437
Epoch 65/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0020 - acc: 0.8742 - rmse: 0.0363 - mse: 0.0020 - r_square: 0.9653 - val_loss: 0.0050 - val_acc: 0.9749 - val_rmse: 0.0523 - val_mse: 0.0050 - val_r_square: 0.9482
Epoch 66/100
35456/35456 [==============================] - 5s 135us/step - loss: 0.0020 - acc: 0.8807 - rmse: 0.0372 - mse: 0.0020 - r_square: 0.9684 - val_loss: 0.0052 - val_acc: 0.9757 - val_rmse: 0.0565 - val_mse: 0.0052 - val_r_square: 0.9444
Epoch 67/100
35456/35456 [==============================] - 5s 135us/step - loss: 0.0019 - acc: 0.8921 - rmse: 0.0355 - mse: 0.0019 - r_square: 0.9652 - val_loss: 0.0049 - val_acc: 0.9751 - val_rmse: 0.0508 - val_mse: 0.0049 - val_r_square: 0.9499
Epoch 68/100
35456/35

35456/35456 [==============================] - 5s 133us/step - loss: 0.0051 - acc: 0.9040 - mse: 0.0051 - rmse: 0.0555 - r_square: 0.8360 - val_loss: 0.0259 - val_acc: 0.7051 - val_mse: 0.0259 - val_rmse: 0.1493 - val_r_square: 0.6062
Epoch 30/100
35456/35456 [==============================] - 5s 133us/step - loss: 0.0053 - acc: 0.9094 - mse: 0.0053 - rmse: 0.0574 - r_square: 0.8382 - val_loss: 0.0271 - val_acc: 0.6745 - val_mse: 0.0271 - val_rmse: 0.1526 - val_r_square: 0.5872
Epoch 31/100
35456/35456 [==============================] - 5s 133us/step - loss: 0.0052 - acc: 0.9109 - mse: 0.0052 - rmse: 0.0565 - r_square: 0.8401 - val_loss: 0.0266 - val_acc: 0.5917 - val_mse: 0.0266 - val_rmse: 0.1516 - val_r_square: 0.6065
Epoch 32/100
35456/35456 [==============================] - 5s 134us/step - loss: 0.0055 - acc: 0.9127 - mse: 0.0055 - rmse: 0.0581 - r_square: 0.8367 - val_loss: 0.0226 - val_acc: 0.9274 - val_mse: 0.0226 - val_rmse: 0.1379 - val_r_square: 0.6624
Epoch 33/100
35456/35

35456/35456 [==============================] - 5s 134us/step - loss: 0.0051 - acc: 0.8714 - mse: 0.0051 - rmse: 0.0564 - r_square: 0.8981 - val_loss: 0.0175 - val_acc: 0.9788 - val_mse: 0.0175 - val_rmse: 0.1213 - val_r_square: 0.7481
Epoch 96/100
35456/35456 [==============================] - 5s 134us/step - loss: 0.0050 - acc: 0.8759 - mse: 0.0050 - rmse: 0.0559 - r_square: 0.9011 - val_loss: 0.0179 - val_acc: 0.9821 - val_mse: 0.0179 - val_rmse: 0.1233 - val_r_square: 0.7437
Epoch 97/100
35456/35456 [==============================] - 5s 134us/step - loss: 0.0048 - acc: 0.8868 - mse: 0.0048 - rmse: 0.0553 - r_square: 0.9057 - val_loss: 0.0177 - val_acc: 0.9842 - val_mse: 0.0177 - val_rmse: 0.1228 - val_r_square: 0.7514
Epoch 98/100
35456/35456 [==============================] - 5s 134us/step - loss: 0.0048 - acc: 0.8832 - mse: 0.0048 - rmse: 0.0547 - r_square: 0.9061 - val_loss: 0.0170 - val_acc: 0.9842 - val_mse: 0.0170 - val_rmse: 0.1198 - val_r_square: 0.7652
Epoch 99/100
35456/35

# visibility

In [17]:
df.keys()
col=[ 'Outbreak',  'Country/Region', 'visibility','Confirmed_x','Deaths','Recovered']
newdf=df[col]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = normalize_data(newdf, scaler,df.shape[1])
x_train, y_train, x_validate, y_validate = load_data(data,time_step=2, after_day=4, validate_percent=0.8)
print('train data: ', x_train.shape, y_train.shape)
print('validate data: ', x_validate.shape, y_validate.shape)
print("*"*50)
print("*"*50)
print("*"*50)
print("LSTM")
input_shape = (2, data.shape[1])
model = base_model(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("Seq2Seq")
input_shape = (2, data.shape[1])
model = seq2seq(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer='adam',metrics=['acc',rmse,mse,r_square])
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
history = model.fit(x_train, y_train, batch_size=256,epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))
print("*"*50)
print("*"*50)
print("*"*50)
print("GRU")
opt = Adam(lr=0.01)
input_shape = (2, data.shape[1])
model = base_model2(data.shape[1], 4, input_shape)
model.compile(loss='mse', optimizer=opt,metrics=['acc',mse,rmse,r_square])
history = model.fit(x_train, y_train, batch_size=64, epochs=100,shuffle=False ,validation_data=(x_validate, y_validate))


total data:  (44320, 6, 6)
train data:  (35456, 2, 6) (35456, 4, 6)
validate data:  (8864, 2, 6) (8864, 4, 6)
**************************************************
**************************************************
**************************************************
LSTM
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 10s 282us/step - loss: 0.0772 - acc: 0.9190 - rmse: 0.2367 - mse: 0.0772 - r_square: 0.3785 - val_loss: 0.0205 - val_acc: 0.8324 - val_rmse: 0.1259 - val_mse: 0.0205 - val_r_square: 0.8644
Epoch 2/100
35456/35456 [==============================] - 4s 103us/step - loss: 0.0137 - acc: 0.9124 - rmse: 0.0985 - mse: 0.0137 - r_square: 0.8937 - val_loss: 0.0101 - val_acc: 0.8313 - val_rmse: 0.0810 - val_mse: 0.0101 - val_r_square: 0.9359
Epoch 3/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0075 - acc: 0.8869 - rmse: 0.0693 - mse: 0.0075 - r_square: 0.9394 - val_loss: 0.0098 - val_acc: 0.8303 - 

Epoch 33/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0041 - acc: 0.8946 - rmse: 0.0437 - mse: 0.0041 - r_square: 0.9670 - val_loss: 0.0059 - val_acc: 0.8328 - val_rmse: 0.0568 - val_mse: 0.0059 - val_r_square: 0.9599
Epoch 34/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0042 - acc: 0.8920 - rmse: 0.0445 - mse: 0.0042 - r_square: 0.9661 - val_loss: 0.0049 - val_acc: 0.8341 - val_rmse: 0.0460 - val_mse: 0.0049 - val_r_square: 0.9678
Epoch 35/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0041 - acc: 0.8979 - rmse: 0.0441 - mse: 0.0041 - r_square: 0.9668 - val_loss: 0.0063 - val_acc: 0.8326 - val_rmse: 0.0599 - val_mse: 0.0063 - val_r_square: 0.9575
Epoch 36/100
35456/35456 [==============================] - 3s 92us/step - loss: 0.0043 - acc: 0.8939 - rmse: 0.0456 - mse: 0.0043 - r_square: 0.9652 - val_loss: 0.0048 - val_acc: 0.8345 - val_rmse: 0.0460 - val_mse: 0.0048 - val_r_square: 0.9685
Epoch 37/100

35456/35456 [==============================] - 3s 93us/step - loss: 0.0034 - acc: 0.8680 - rmse: 0.0376 - mse: 0.0034 - r_square: 0.9718 - val_loss: 0.0049 - val_acc: 0.8284 - val_rmse: 0.0452 - val_mse: 0.0049 - val_r_square: 0.9681
Epoch 100/100
35456/35456 [==============================] - 3s 93us/step - loss: 0.0034 - acc: 0.8790 - rmse: 0.0359 - mse: 0.0034 - r_square: 0.9723 - val_loss: 0.0051 - val_acc: 0.8408 - val_rmse: 0.0464 - val_mse: 0.0051 - val_r_square: 0.9671
**************************************************
**************************************************
**************************************************
Seq2Seq
Train on 35456 samples, validate on 8864 samples
Epoch 1/100
35456/35456 [==============================] - 11s 314us/step - loss: 0.0480 - acc: 0.9206 - rmse: 0.1678 - mse: 0.0480 - r_square: 0.6208 - val_loss: 0.0100 - val_acc: 0.8311 - val_rmse: 0.0848 - val_mse: 0.0100 - val_r_square: 0.9349
Epoch 2/100
35456/35456 [==============================] - 5

35456/35456 [==============================] - 5s 136us/step - loss: 0.0038 - acc: 0.8722 - rmse: 0.0413 - mse: 0.0038 - r_square: 0.9689 - val_loss: 0.0061 - val_acc: 0.8324 - val_rmse: 0.0597 - val_mse: 0.0061 - val_r_square: 0.9591
Epoch 65/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0042 - acc: 0.8918 - rmse: 0.0457 - mse: 0.0042 - r_square: 0.9653 - val_loss: 0.0087 - val_acc: 0.8324 - val_rmse: 0.0771 - val_mse: 0.0087 - val_r_square: 0.9407
Epoch 66/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0040 - acc: 0.8998 - rmse: 0.0434 - mse: 0.0040 - r_square: 0.9672 - val_loss: 0.0063 - val_acc: 0.8322 - val_rmse: 0.0618 - val_mse: 0.0063 - val_r_square: 0.9577
Epoch 67/100
35456/35456 [==============================] - 5s 140us/step - loss: 0.0040 - acc: 0.9011 - rmse: 0.0439 - mse: 0.0040 - r_square: 0.9674 - val_loss: 0.0066 - val_acc: 0.8296 - val_rmse: 0.0631 - val_mse: 0.0066 - val_r_square: 0.9552
Epoch 68/100
35456/35

35456/35456 [==============================] - 5s 137us/step - loss: 0.0051 - acc: 0.8968 - mse: 0.0051 - rmse: 0.0507 - r_square: 0.9569 - val_loss: 0.0133 - val_acc: 0.8324 - val_mse: 0.0133 - val_rmse: 0.0981 - val_r_square: 0.9132
Epoch 30/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0047 - acc: 0.8952 - mse: 0.0047 - rmse: 0.0476 - r_square: 0.9604 - val_loss: 0.0121 - val_acc: 0.8324 - val_mse: 0.0121 - val_rmse: 0.0939 - val_r_square: 0.9201
Epoch 31/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0045 - acc: 0.8942 - mse: 0.0045 - rmse: 0.0461 - r_square: 0.9619 - val_loss: 0.0122 - val_acc: 0.8324 - val_mse: 0.0122 - val_rmse: 0.0943 - val_r_square: 0.9197
Epoch 32/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0048 - acc: 0.8967 - mse: 0.0048 - rmse: 0.0486 - r_square: 0.9594 - val_loss: 0.0118 - val_acc: 0.8324 - val_mse: 0.0118 - val_rmse: 0.0928 - val_r_square: 0.9220
Epoch 33/100
35456/35

35456/35456 [==============================] - 5s 136us/step - loss: 0.0044 - acc: 0.8923 - mse: 0.0044 - rmse: 0.0446 - r_square: 0.9627 - val_loss: 0.0113 - val_acc: 0.8356 - val_mse: 0.0113 - val_rmse: 0.0871 - val_r_square: 0.9255
Epoch 96/100
35456/35456 [==============================] - 5s 136us/step - loss: 0.0045 - acc: 0.8887 - mse: 0.0045 - rmse: 0.0452 - r_square: 0.9618 - val_loss: 0.0118 - val_acc: 0.8340 - val_mse: 0.0118 - val_rmse: 0.0897 - val_r_square: 0.9222
Epoch 97/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0044 - acc: 0.8927 - mse: 0.0044 - rmse: 0.0447 - r_square: 0.9624 - val_loss: 0.0111 - val_acc: 0.8373 - val_mse: 0.0111 - val_rmse: 0.0862 - val_r_square: 0.9264
Epoch 98/100
35456/35456 [==============================] - 5s 137us/step - loss: 0.0044 - acc: 0.8840 - mse: 0.0044 - rmse: 0.0438 - r_square: 0.9630 - val_loss: 0.0111 - val_acc: 0.8376 - val_mse: 0.0111 - val_rmse: 0.0865 - val_r_square: 0.9264
Epoch 99/100
35456/35